In [1]:
import os
import pickle
import pandas as pd
import numpy as np
from scipy.stats import stats
import statistics as st
import random

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

# spin-test permutations
from netneurotools import datasets, freesurfer, stats
from nibabel.freesurfer import read_annot, read_geometry
def _decode_list(vals):
    """ List decoder
    """
    return [val.decode() if hasattr(val, 'decode') else val for val in vals]

import warnings
warnings.filterwarnings("ignore")

/Users/zhangxihan/.local/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
## set dirs
base_dir = '/Volumes/GoogleDrive/My Drive/Gradient_Shift_Cellular_Basis/Milgram'
gradient_dir = base_dir+'/data/Gradients_Margulies2016'
figure_dir = base_dir+'/figures/NetworkEnrichment'
net_enrich_dir = base_dir+'/data/NetworkEnrichment'

In [3]:
## 1. load the 18 cell-types
LakeDFC_schaefer400 = pd.read_csv(gradient_dir+'/schaeffer_LAKE_DFC_400_7Net_expr_mat_new_NormZscore0.3.csv',header=None,index_col=0)
LakeVIS_schaefer400 = pd.read_csv(gradient_dir+'/schaeffer_LAKE_VIS_400_7Net_expr_mat_new_NormZscore0.3.csv',header=0,index_col=0)
# name list of row names (cell-type)
cell_types_DFC = list(LakeDFC_schaefer400.index.values)
cell_types_VIS = list(LakeVIS_schaefer400.index.values)
# drop NaN columns in empirical cell type fractions
LakeDFC_schaefer400_drop = LakeDFC_schaefer400.dropna(axis=1)
LakeVIS_schaefer400_drop = LakeVIS_schaefer400.dropna(axis=1)
# 0~190: LH
# 191~339: RH
# remove the Ex2 from DFC and In2 from VIS
LakeDFC_schaefer400_drop = LakeDFC_schaefer400_drop.drop(index=('DFC_Ex2'))
LakeVIS_schaefer400_drop = LakeVIS_schaefer400_drop.drop(index=('VIS_In2'))
# extract the values
LakeDFC_schaefer400_mat = LakeDFC_schaefer400_drop.to_numpy()
LakeVIS_schaefer400_mat = LakeVIS_schaefer400_drop.to_numpy()

## 2. load the labels for the Schaefer networks
schaefer400_7Networks = pd.read_csv(gradient_dir+'/schaefer_400_7Net_labels_manually_added.csv',header=None,index_col=0)
# dropped version of it
schaefer400_7Networks_drop = pd.read_csv(net_enrich_dir + '/schaefer400_7Networks_NaNDropped.csv',header=0,index_col=0)
schaefer400_7Networks_mat = schaefer400_7Networks_drop.to_numpy()


## 3. load the permutated network labels
Null_7NetLabel = pd.read_csv(net_enrich_dir + '/1000Null_NetLabel_Schaefer400_NaNDropped.csv',header=0,index_col=0)
Null_7NetLabel_mat = Null_7NetLabel.to_numpy()

## Part A: Seperate Lake DFC and Lake VIS
The calssifiers were trained from Lake DFC and Lake VIS separately.
### Part A1: Left Hemisphere Only
Including parcels from only the left hemisphere.
#### 1000 Classifiers
The classifiers were trained from real data.

In [16]:
clf_num = 1000
parcel_num = 191
test_parcel_num = 58
net_num = 7
schaefer400_7Networks_mat_left = schaefer400_7Networks_mat[:191,:]
LakeDFC_schaefer400_mat_left = LakeDFC_schaefer400_mat[:,:191]
LakeVIS_schaefer400_mat_left = LakeVIS_schaefer400_mat[:,:191]

clf_test = np.zeros((test_parcel_num,clf_num))
sample_test = np.zeros((test_parcel_num,clf_num))

clf_pred_DFC = np.zeros((test_parcel_num,clf_num))
clf_score_DFC = []
clf_recall_DFC = np.zeros((clf_num,net_num))
C_best_DFC = []
gamma_best_DFC = []
kernel_best_DFC = []

clf_pred_VIS = np.zeros((test_parcel_num,clf_num))
clf_score_VIS = []
clf_recall_VIS = np.zeros((clf_num,net_num))
C_best_VIS = []
gamma_best_VIS = []
kernel_best_VIS = []
for c in range(clf_num):
    # Step1: Split 70% for train and 30% for test
    train, test = train_test_split(list(range(parcel_num)), test_size=0.3, stratify=schaefer400_7Networks_mat_left)
    label_train = schaefer400_7Networks_mat_left[train]
    label_test = schaefer400_7Networks_mat_left[test]
    # save the real category of the tested parcels
    clf_test[:,c] = label_test[:,0]
    # save the ID of the tested parcels
    sample_test[:,c] = test

    LakeDFC_train = np.transpose(LakeDFC_schaefer400_mat_left[:,train])
    LakeDFC_test = np.transpose(LakeDFC_schaefer400_mat_left[:,test])

    LakeVIS_train = np.transpose(LakeVIS_schaefer400_mat_left[:,train])
    LakeVIS_test = np.transpose(LakeVIS_schaefer400_mat_left[:,test])
    
    
    ### Lake DFC ###
    # Step2: Nested 3-fold cross-validation for hyperparameter tunning
    # Outter loop: test the classifier performance
    kf_outer = StratifiedKFold(n_splits=3)
    clf_score = []
    C_best = []
    gamma_best = []
    kernel_best = []
    for train_outer, test_outer in kf_outer.split(LakeDFC_train, label_train):
        # Pull out the sample data
        label_train_outer = label_train[train_outer]
        label_test_outer = label_train[test_outer]
    
        DFC_train_outer = LakeDFC_train[train_outer,:]
        DFC_test_outer = LakeDFC_train[test_outer,:]

        ###### Inner loop: tune the hyper-parameter ######
        kf_inner = StratifiedKFold(n_splits=2)
    
        # Search over different regularization parameters: smaller values specify stronger regularization.
        parameters = {'C':[0.1,10,100],
                      'gamma':[0.001, 1, 1000],
                      'kernel':['poly', 'rbf']}
    
        inner_clf = GridSearchCV(
            SVC(class_weight='balanced'),
            parameters,
            cv=kf_inner,
            scoring='f1_macro',
            return_train_score=True)
        inner_clf.fit(DFC_train_outer, label_train_outer)
        ###### Inner loop: tune the hyper-parameter ######
    
        # Find the best hyperparameter
        C_best.append(inner_clf.best_params_['C'])
        gamma_best.append(inner_clf.best_params_['gamma'])
        kernel_best.append(inner_clf.best_params_['kernel'])
    
        # Train the classifier with the best hyperparameter using training and validation set
        classifier = SVC(class_weight='balanced',
                         C=inner_clf.best_params_['C'],
                         gamma=inner_clf.best_params_['gamma'],
                         kernel=inner_clf.best_params_['kernel'])
        outer_clf = classifier.fit(DFC_train_outer, label_train_outer)
        
        # Validate the classifier
        score = outer_clf.score(DFC_test_outer, label_test_outer)
        clf_score.append(score)
    
    # find the best hyper-combo
    best_idx = clf_score.index(max(clf_score))
    
    C_best_DFC.append(C_best[best_idx])
    gamma_best_DFC.append(gamma_best[best_idx])
    kernel_best_DFC.append(kernel_best[best_idx])
    print ('Best cost value:', C_best_DFC[c])
    print ('Best gamma:', gamma_best_DFC[c])
    print ('Best kernel:', kernel_best_DFC[c])
    
    classifier = SVC(class_weight='balanced',
                     C=C_best[best_idx],
                     gamma=gamma_best[best_idx],
                     kernel=kernel_best[best_idx])
    final_clf = classifier.fit(LakeDFC_train, label_train)
    
    # Test the classifier
    clf_pred_DFC[:,c] = final_clf.predict(LakeDFC_test)
    clf_score_DFC.append(f1_score(label_test, clf_pred_DFC[:,c], average='macro'))
    print(str(c)+' f-1 score for Lake DFC is: '+str(clf_score_DFC[c]))
    clf_recall_DFC[c,:] = recall_score(label_test, clf_pred_DFC[:,c], average=None)

    
    ### Lake VIS ###
    # Step2: Nested 3-fold cross-validation for hyperparameter tunning
    # Outter loop: test the classifier performance
    kf_outer = StratifiedKFold(n_splits=3)
    clf_score = []
    C_best = []
    gamma_best = []
    kernel_best = []
    for train_outer, test_outer in kf_outer.split(LakeVIS_train, label_train):
        # Pull out the sample data
        label_train_outer = label_train[train_outer]
        label_test_outer = label_train[test_outer]
    
        VIS_train_outer = LakeVIS_train[train_outer,:]
        VIS_test_outer = LakeVIS_train[test_outer,:]

        ###### Inner loop: tune the hyper-parameter ######
        kf_inner = StratifiedKFold(n_splits=3)
    
        # Search over different regularization parameters: smaller values specify stronger regularization.
        parameters = {'C':[0.1,10,100],
                      'gamma':[0.001, 1, 1000],
                      'kernel':['poly', 'rbf']}
    
        inner_clf = GridSearchCV(
            SVC(class_weight='balanced'),
            parameters,
            cv=kf_inner,
            scoring='f1_macro',
            return_train_score=True)
        inner_clf.fit(VIS_train_outer, label_train_outer)
        ###### Inner loop: tune the hyper-parameter ######
    
        # Find the best hyperparameter
        C_best.append(inner_clf.best_params_['C'])
        gamma_best.append(inner_clf.best_params_['gamma'])
        kernel_best.append(inner_clf.best_params_['kernel'])
    
        # Train the classifier with the best hyperparameter using training and validation set
        classifier = SVC(class_weight='balanced',
                         C=inner_clf.best_params_['C'],
                         gamma=inner_clf.best_params_['gamma'],
                         kernel=inner_clf.best_params_['kernel'])
        outer_clf = classifier.fit(VIS_train_outer, label_train_outer)
        
        # Validate the classifier
        score = outer_clf.score(VIS_test_outer, label_test_outer)
        clf_score.append(score)
    
    # find the best hyper-combo
    best_idx = clf_score.index(max(clf_score))
    
    C_best_VIS.append(C_best[best_idx])
    gamma_best_VIS.append(gamma_best[best_idx])
    kernel_best_VIS.append(kernel_best[best_idx])
    print ('Best cost value:', C_best_VIS[c])
    print ('Best gamma:', gamma_best_VIS[c])
    print ('Best kernel:', kernel_best_VIS[c])
    
    classifier = SVC(class_weight='balanced',
                     C=C_best[best_idx],
                     gamma=gamma_best[best_idx],
                     kernel=kernel_best[best_idx])
    final_clf = classifier.fit(LakeVIS_train, label_train)
    
    # Test the classifier
    clf_pred_VIS[:,c] = final_clf.predict(LakeVIS_test)
    clf_score_VIS.append(f1_score(label_test, clf_pred_VIS[:,c], average='macro'))
    print(str(c)+' f-1 score for Lake VIS is: '+str(clf_score_VIS[c]))
    clf_recall_VIS[c,:] = recall_score(label_test, clf_pred_VIS[:,c], average=None)

Best cost value: 0.1
Best gamma: 1000
Best kernel: poly
0 f-1 score for Lake DFC is: 0.32944471494891664
Best cost value: 100
Best gamma: 1
Best kernel: rbf
0 f-1 score for Lake VIS is: 0.4018135074035695
Best cost value: 100
Best gamma: 1
Best kernel: rbf
1 f-1 score for Lake DFC is: 0.3969651263768911
Best cost value: 100
Best gamma: 1
Best kernel: rbf
1 f-1 score for Lake VIS is: 0.44297565179918114
Best cost value: 10
Best gamma: 1
Best kernel: rbf
2 f-1 score for Lake DFC is: 0.36156343656343654
Best cost value: 10
Best gamma: 1
Best kernel: rbf
2 f-1 score for Lake VIS is: 0.37233147804576383
Best cost value: 100
Best gamma: 1
Best kernel: poly
3 f-1 score for Lake DFC is: 0.34854086562161096
Best cost value: 100
Best gamma: 1
Best kernel: rbf
3 f-1 score for Lake VIS is: 0.37648797710717213
Best cost value: 100
Best gamma: 1
Best kernel: rbf
4 f-1 score for Lake DFC is: 0.39801587301587305


KeyboardInterrupt: 

In [130]:
# save the result
with open(net_enrich_dir+'/1000ClassifiersBalanced_Left.pkl', 'wb') as f:
    pickle.dump([clf_test, sample_test,
                 clf_pred_DFC, clf_score_DFC, clf_recall_DFC, C_best_DFC, gamma_best_DFC, kernel_best_DFC,
                 clf_pred_VIS, clf_score_VIS, clf_recall_VIS, C_best_VIS, gamma_best_VIS, kernel_best_VIS], f)
# load the results
#clf_test, sample_test, clf_pred_DFC, clf_score_DFC, clf_recall_DFC, C_best_DFC, gamma_best_DFC, kernel_best_DFC, clf_pred_VIS, clf_score_VIS, clf_recall_VIS, C_best_VIS, gamma_best_VIS, kernel_best_VIS = pickle.load(open(net_enrich_dir+'/1000Classifiers_Left.pkl',"rb"))

#### 1000 nulls
- In the comment `# Step0: Permutate the labels`, is the way to generate null data by permutating the network lables totally at random.
- The uncommented `# Step0: Get the spin-tested label`, is the way to generate null data by spinning the network lables controllig for spatial autocorrelation among parcels.

In the paper, we did both to generate p-values at different stringent level.

In [160]:
clf_num = 1000
parcel_num = 191
test_parcel_num = 58
net_num = 7
schaefer400_7Networks_mat_left = schaefer400_7Networks_mat[:191,:]
LakeDFC_schaefer400_mat_left = LakeDFC_schaefer400_mat[:,:191]
LakeVIS_schaefer400_mat_left = LakeVIS_schaefer400_mat[:,:191]

# Step0: Permutate the labels
#schaefer400_7Networks_perm = np.random.permutation(schaefer400_7Networks_mat_left)
#schaefer400_7Networks_perm = np.random.randint(low=1, high=8, size=parcel_num)

null_test = np.zeros((test_parcel_num,clf_num))
sample_test = np.zeros((test_parcel_num,clf_num))

null_pred_DFC = np.zeros((test_parcel_num,clf_num))
null_score_DFC = []
null_recall_DFC = np.zeros((clf_num,net_num))

null_pred_VIS = np.zeros((test_parcel_num,clf_num))
null_score_VIS = []
null_recall_VIS = np.zeros((clf_num,net_num))
for c in range(clf_num):
    
    # Step0: Get the spin-tested label
    schaefer400_7Networks_perm = Null_7NetLabel_mat[:191,c]

    # Step1: Split 70% for train and 30% for test
    train, test = train_test_split(list(range(parcel_num)), test_size=0.3, stratify=schaefer400_7Networks_perm)
    label_train = schaefer400_7Networks_perm[train]
    label_test = schaefer400_7Networks_perm[test]
    # save the permutated lable
    #null_test[:,c] = label_test[:,0]
    null_test[:,c] = label_test
    # save the ID of the tested parcels
    sample_test[:,c] = test

    LakeDFC_train = np.transpose(LakeDFC_schaefer400_mat_left[:,train])
    LakeDFC_test = np.transpose(LakeDFC_schaefer400_mat_left[:,test])

    LakeVIS_train = np.transpose(LakeVIS_schaefer400_mat_left[:,train])
    LakeVIS_test = np.transpose(LakeVIS_schaefer400_mat_left[:,test])


    ### Lake DFC ###
    # Step2: Random draw a hyper-combo then train the classifer
    # find the best hyper-combo
    best_idx = np.random.randint(1000, size=1)[0]

    # Step3: Train the classifier
    classifier = SVC(class_weight='balanced',C=C_best_DFC[best_idx],gamma=gamma_best_DFC[best_idx],kernel=kernel_best_DFC[best_idx])
    final_clf = classifier.fit(LakeDFC_train, label_train)

    # Test the classifier
    null_pred_DFC[:,c] = final_clf.predict(LakeDFC_test)
    null_score_DFC.append(f1_score(label_test, null_pred_DFC[:,c], average='macro'))
    print(str(c)+' f-1 score for Lake DFC is: '+str(null_score_DFC[c]))
    null_recall_DFC[c,:] = recall_score(label_test, null_pred_DFC[:,c], average=None)


    ### Lake VIS ###
    # Step2: Random draw a hyper-combo then train the classifer
    # find the best hyper-combo
    best_idx = np.random.randint(1000, size=1)[0]


    # Step3: Train the classifier
    classifier = SVC(class_weight='balanced',C=C_best_VIS[best_idx],gamma=gamma_best_VIS[best_idx],kernel=kernel_best_VIS[best_idx])
    final_clf = classifier.fit(LakeVIS_train, label_train)

    # Test the classifier
    null_pred_VIS[:,c] = final_clf.predict(LakeVIS_test)
    null_score_VIS.append(f1_score(label_test, null_pred_VIS[:,c], average='macro'))
    print(str(c)+' f-1 score for Lake VIS is: '+str(null_score_VIS[c]))
    null_recall_VIS[c,:] = recall_score(label_test, null_pred_VIS[:,c], average=None)

0 f-1 score for Lake DFC is: 0.28015873015873016
0 f-1 score for Lake VIS is: 0.26821789321789324
1 f-1 score for Lake DFC is: 0.3203257060399918
1 f-1 score for Lake VIS is: 0.3657802514945372
2 f-1 score for Lake DFC is: 0.13170426065162907
2 f-1 score for Lake VIS is: 0.14574777074777076
3 f-1 score for Lake DFC is: 0.2589058974152142
3 f-1 score for Lake VIS is: 0.21981721981721977
4 f-1 score for Lake DFC is: 0.15064181849896133
4 f-1 score for Lake VIS is: 0.17501046572475146
5 f-1 score for Lake DFC is: 0.3242819981950417
5 f-1 score for Lake VIS is: 0.34581347224204373
6 f-1 score for Lake DFC is: 0.22537710736468503
6 f-1 score for Lake VIS is: 0.25893468498510513
7 f-1 score for Lake DFC is: 0.24292766548405645
7 f-1 score for Lake VIS is: 0.1970969089390142
8 f-1 score for Lake DFC is: 0.3045603138280484
8 f-1 score for Lake VIS is: 0.3040336134453781
9 f-1 score for Lake DFC is: 0.2518010393950244
9 f-1 score for Lake VIS is: 0.21413354815610455
10 f-1 score for Lake DFC is

97 f-1 score for Lake DFC is: 0.2658724733912704
97 f-1 score for Lake VIS is: 0.2889588223953549
98 f-1 score for Lake DFC is: 0.2975030145837599
98 f-1 score for Lake VIS is: 0.2580324437467295
99 f-1 score for Lake DFC is: 0.29936394015341383
99 f-1 score for Lake VIS is: 0.24004884004884003
100 f-1 score for Lake DFC is: 0.20570517983357892
100 f-1 score for Lake VIS is: 0.20787589464060052
101 f-1 score for Lake DFC is: 0.28349162914380305
101 f-1 score for Lake VIS is: 0.20109176537747966
102 f-1 score for Lake DFC is: 0.13863066757803602
102 f-1 score for Lake VIS is: 0.16831694621168308
103 f-1 score for Lake DFC is: 0.26964480799067264
103 f-1 score for Lake VIS is: 0.22040816326530613
104 f-1 score for Lake DFC is: 0.1354978354978355
104 f-1 score for Lake VIS is: 0.2067790594498669
105 f-1 score for Lake DFC is: 0.20665445665445667
105 f-1 score for Lake VIS is: 0.20172735760971056
106 f-1 score for Lake DFC is: 0.279385693671408
106 f-1 score for Lake VIS is: 0.149312592169

195 f-1 score for Lake DFC is: 0.17188208616780049
195 f-1 score for Lake VIS is: 0.3035905271199389
196 f-1 score for Lake DFC is: 0.3455975102591644
196 f-1 score for Lake VIS is: 0.2639548126943085
197 f-1 score for Lake DFC is: 0.22444451464796442
197 f-1 score for Lake VIS is: 0.17207792207792208
198 f-1 score for Lake DFC is: 0.32388501945687637
198 f-1 score for Lake VIS is: 0.26235708735708735
199 f-1 score for Lake DFC is: 0.2592378381852066
199 f-1 score for Lake VIS is: 0.23089235694277707
200 f-1 score for Lake DFC is: 0.2540416976507202
200 f-1 score for Lake VIS is: 0.2572741377252656
201 f-1 score for Lake DFC is: 0.27432216905901113
201 f-1 score for Lake VIS is: 0.23879127408539175
202 f-1 score for Lake DFC is: 0.2817472737549464
202 f-1 score for Lake VIS is: 0.1959538999012683
203 f-1 score for Lake DFC is: 0.3243476701025237
203 f-1 score for Lake VIS is: 0.31070155334861216
204 f-1 score for Lake DFC is: 0.27738095238095234
204 f-1 score for Lake VIS is: 0.2789115

295 f-1 score for Lake DFC is: 0.33619244799369025
295 f-1 score for Lake VIS is: 0.2544502281344387
296 f-1 score for Lake DFC is: 0.18040502914452494
296 f-1 score for Lake VIS is: 0.18166539343009935
297 f-1 score for Lake DFC is: 0.1803617641388539
297 f-1 score for Lake VIS is: 0.25192450406736117
298 f-1 score for Lake DFC is: 0.21362018491607168
298 f-1 score for Lake VIS is: 0.4387154861944778
299 f-1 score for Lake DFC is: 0.15863182711008797
299 f-1 score for Lake VIS is: 0.16013510299224581
300 f-1 score for Lake DFC is: 0.28416361760015013
300 f-1 score for Lake VIS is: 0.26301467955603297
301 f-1 score for Lake DFC is: 0.33418640183346066
301 f-1 score for Lake VIS is: 0.3168831168831169
302 f-1 score for Lake DFC is: 0.1938023088023088
302 f-1 score for Lake VIS is: 0.22804232804232807
303 f-1 score for Lake DFC is: 0.17515262515262514
303 f-1 score for Lake VIS is: 0.23145363408521305
304 f-1 score for Lake DFC is: 0.22875877623776786
304 f-1 score for Lake VIS is: 0.267

393 f-1 score for Lake VIS is: 0.3357015533486121
394 f-1 score for Lake DFC is: 0.11877562619250599
394 f-1 score for Lake VIS is: 0.1739779932391897
395 f-1 score for Lake DFC is: 0.16715728715728714
395 f-1 score for Lake VIS is: 0.17362576401478003
396 f-1 score for Lake DFC is: 0.25562182915124093
396 f-1 score for Lake VIS is: 0.19238811313999285
397 f-1 score for Lake DFC is: 0.18617250196197568
397 f-1 score for Lake VIS is: 0.27362858558510733
398 f-1 score for Lake DFC is: 0.2960776123314823
398 f-1 score for Lake VIS is: 0.31983808485548765
399 f-1 score for Lake DFC is: 0.15619763694951663
399 f-1 score for Lake VIS is: 0.23886145980812934
400 f-1 score for Lake DFC is: 0.3342336934773909
400 f-1 score for Lake VIS is: 0.36194056839218136
401 f-1 score for Lake DFC is: 0.25343231931467225
401 f-1 score for Lake VIS is: 0.22000690131124914
402 f-1 score for Lake DFC is: 0.2730364873222016
402 f-1 score for Lake VIS is: 0.269984485022079
403 f-1 score for Lake DFC is: 0.30390

...

587 f-1 score for Lake VIS is: 0.23907045335616764
588 f-1 score for Lake DFC is: 0.3316974950515323
588 f-1 score for Lake VIS is: 0.2822204266321914
589 f-1 score for Lake DFC is: 0.24622515256709437
589 f-1 score for Lake VIS is: 0.25967532467532467
590 f-1 score for Lake DFC is: 0.1712305475549904
590 f-1 score for Lake VIS is: 0.12266380236305048
591 f-1 score for Lake DFC is: 0.1823169267707083
591 f-1 score for Lake VIS is: 0.15246975246975247
592 f-1 score for Lake DFC is: 0.24343434343434342
592 f-1 score for Lake VIS is: 0.24978398376894617
593 f-1 score for Lake DFC is: 0.21132195568285791
593 f-1 score for Lake VIS is: 0.19149967679379443
594 f-1 score for Lake DFC is: 0.37159709915127875
594 f-1 score for Lake VIS is: 0.261512297226583
595 f-1 score for Lake DFC is: 0.22007168458781362
595 f-1 score for Lake VIS is: 0.295010529464311
596 f-1 score for Lake DFC is: 0.24093144950287806
596 f-1 score for Lake VIS is: 0.24024868695275428
597 f-1 score for Lake DFC is: 0.183542

684 f-1 score for Lake VIS is: 0.21034776483692894
685 f-1 score for Lake DFC is: 0.19909297052154198
685 f-1 score for Lake VIS is: 0.2117794486215539
686 f-1 score for Lake DFC is: 0.28932496075353215
686 f-1 score for Lake VIS is: 0.3823181580324438
687 f-1 score for Lake DFC is: 0.2602134707397865
687 f-1 score for Lake VIS is: 0.20295566502463053
688 f-1 score for Lake DFC is: 0.38572825024437923
688 f-1 score for Lake VIS is: 0.16546614186365738
689 f-1 score for Lake DFC is: 0.10786536836957004
689 f-1 score for Lake VIS is: 0.2013854512056539
690 f-1 score for Lake DFC is: 0.2320053338474391
690 f-1 score for Lake VIS is: 0.3320738440303658
691 f-1 score for Lake DFC is: 0.26223776223776224
691 f-1 score for Lake VIS is: 0.26447758590615733
692 f-1 score for Lake DFC is: 0.17900838068020733
692 f-1 score for Lake VIS is: 0.26623396097080315
693 f-1 score for Lake DFC is: 0.1934269434269434
693 f-1 score for Lake VIS is: 0.23870688626327724
694 f-1 score for Lake DFC is: 0.21776

780 f-1 score for Lake VIS is: 0.19421768707482995
781 f-1 score for Lake DFC is: 0.3308450652988468
781 f-1 score for Lake VIS is: 0.28070130927273784
782 f-1 score for Lake DFC is: 0.19484242465608922
782 f-1 score for Lake VIS is: 0.19717398070339248
783 f-1 score for Lake DFC is: 0.2858432261793606
783 f-1 score for Lake VIS is: 0.26081822593619325
784 f-1 score for Lake DFC is: 0.29092637054821935
784 f-1 score for Lake VIS is: 0.2765037593984963
785 f-1 score for Lake DFC is: 0.19812641298398417
785 f-1 score for Lake VIS is: 0.3841346178188284
786 f-1 score for Lake DFC is: 0.23075593873913203
786 f-1 score for Lake VIS is: 0.17812865497076022
787 f-1 score for Lake DFC is: 0.3203007518796993
787 f-1 score for Lake VIS is: 0.3528159089722846
788 f-1 score for Lake DFC is: 0.1536429064379375
788 f-1 score for Lake VIS is: 0.21556732740943266
789 f-1 score for Lake DFC is: 0.30628641200069767
789 f-1 score for Lake VIS is: 0.2592370281445912
790 f-1 score for Lake DFC is: 0.151858

878 f-1 score for Lake VIS is: 0.1968235806908576
879 f-1 score for Lake DFC is: 0.3036764356092087
879 f-1 score for Lake VIS is: 0.2071761145929944
880 f-1 score for Lake DFC is: 0.2824913182056039
880 f-1 score for Lake VIS is: 0.3301012712777419
881 f-1 score for Lake DFC is: 0.24291979949874687
881 f-1 score for Lake VIS is: 0.19422586425874697
882 f-1 score for Lake DFC is: 0.2493271139887681
882 f-1 score for Lake VIS is: 0.1827315541601256
883 f-1 score for Lake DFC is: 0.3099780826488901
883 f-1 score for Lake VIS is: 0.3116823652537938
884 f-1 score for Lake DFC is: 0.23659673659673658
884 f-1 score for Lake VIS is: 0.21805704738035567
885 f-1 score for Lake DFC is: 0.22929393979814147
885 f-1 score for Lake VIS is: 0.16085462525091007
886 f-1 score for Lake DFC is: 0.2844536222977255
886 f-1 score for Lake VIS is: 0.2860221011481516
887 f-1 score for Lake DFC is: 0.24388169039198787
887 f-1 score for Lake VIS is: 0.2879818594104308
888 f-1 score for Lake DFC is: 0.3886744834

973 f-1 score for Lake DFC is: 0.22480091855253898
973 f-1 score for Lake VIS is: 0.2144927536231884
974 f-1 score for Lake DFC is: 0.32048928883294214
974 f-1 score for Lake VIS is: 0.3290037754232128
975 f-1 score for Lake DFC is: 0.17907456769012833
975 f-1 score for Lake VIS is: 0.23495552066980638
976 f-1 score for Lake DFC is: 0.21029258098223616
976 f-1 score for Lake VIS is: 0.28543024227234753
977 f-1 score for Lake DFC is: 0.17762732623413738
977 f-1 score for Lake VIS is: 0.3068653568653569
978 f-1 score for Lake DFC is: 0.3201882151461984
978 f-1 score for Lake VIS is: 0.3153369849964277
979 f-1 score for Lake DFC is: 0.32289055973266506
979 f-1 score for Lake VIS is: 0.3513639868903026
980 f-1 score for Lake DFC is: 0.22031988025776847
980 f-1 score for Lake VIS is: 0.25870558013415157
981 f-1 score for Lake DFC is: 0.3538666688571812
981 f-1 score for Lake VIS is: 0.37921036357126575
982 f-1 score for Lake DFC is: 0.23863975913665356
982 f-1 score for Lake VIS is: 0.22531

In [161]:
# save the results
with open(net_enrich_dir+'/1000NullsBalancedSpin_Left.pkl', 'wb') as f:
    pickle.dump([null_test, sample_test,
                 null_pred_DFC, null_score_DFC, null_recall_DFC,
                 null_pred_VIS, null_score_VIS, null_recall_VIS], f)

# load the results
#null_test, sample_test, null_pred_DFC, null_score_DFC, null_recall_DFC, null_pred_VIS, null_score_VIS, null_recall_VIS = pickle.load(open(net_enrich_dir+'/1000Nulls_Left.pkl',"rb"))

### Part A2: Both Hemispheres
Including parcels from both the left and right hemispheres. Note that the right hemisphere is not mirrored to left hemisphere, because the two hemispheres are not purely symmetric.
#### 1000 classifiers

In [133]:
clf_num = 1000
parcel_num = 339
test_parcel_num = 102
net_num = 7

clf_test = np.zeros((test_parcel_num,clf_num))
sample_test = np.zeros((test_parcel_num,clf_num))

clf_pred_DFC = np.zeros((test_parcel_num,clf_num))
clf_score_DFC = []
clf_recall_DFC = np.zeros((clf_num,net_num))
C_best_DFC = []
gamma_best_DFC = []
kernel_best_DFC = []

clf_pred_VIS = np.zeros((test_parcel_num,clf_num))
clf_score_VIS = []
clf_recall_VIS = np.zeros((clf_num,net_num))
C_best_VIS = []
gamma_best_VIS = []
kernel_best_VIS = []
for c in range(clf_num):
    
    # Step1: Split 70% for train and 30% for test
    train, test = train_test_split(list(range(339)), test_size=0.3, stratify=schaefer400_7Networks_mat)
    label_train = schaefer400_7Networks_mat[train]
    label_test = schaefer400_7Networks_mat[test]
    # save the real category of the tested parcels
    clf_test[:,c] = label_test[:,0]
    # save the ID of the tested parcels
    sample_test[:,c] = test

    LakeDFC_train = np.transpose(LakeDFC_schaefer400_mat[:,train])
    LakeDFC_test = np.transpose(LakeDFC_schaefer400_mat[:,test])

    LakeVIS_train = np.transpose(LakeVIS_schaefer400_mat[:,train])
    LakeVIS_test = np.transpose(LakeVIS_schaefer400_mat[:,test])
    
    
    ### Lake DFC ###
    # Step2: Nested 3-fold cross-validation for hyperparameter tunning
    # Outter loop: test the classifier performance
    kf_outer = StratifiedKFold(n_splits=3)
    clf_score = []
    C_best = []
    gamma_best = []
    kernel_best = []
    for train_outer, test_outer in kf_outer.split(LakeDFC_train, label_train):
        # Pull out the sample data
        label_train_outer = label_train[train_outer]
        label_test_outer = label_train[test_outer]
    
        DFC_train_outer = LakeDFC_train[train_outer,:]
        DFC_test_outer = LakeDFC_train[test_outer,:]

        ###### Inner loop: tune the hyper-parameter ######
        kf_inner = StratifiedKFold(n_splits=3)
    
        # Search over different regularization parameters: smaller values specify stronger regularization.
        parameters = {'C':[0.1,10,100],
                      'gamma':[0.001, 1, 1000],
                      'kernel':['poly', 'rbf']}
    
        inner_clf = GridSearchCV(
            SVC(class_weight='balanced'),
            parameters,
            cv=kf_inner,
            scoring='f1_macro',
            return_train_score=True)
        inner_clf.fit(DFC_train_outer, label_train_outer)
        ###### Inner loop: tune the hyper-parameter ######
    
        # Find the best hyperparameter
        C_best.append(inner_clf.best_params_['C'])
        gamma_best.append(inner_clf.best_params_['gamma'])
        kernel_best.append(inner_clf.best_params_['kernel'])
    
        # Train the classifier with the best hyperparameter using training and validation set
        classifier = SVC(class_weight='balanced',
                         C=inner_clf.best_params_['C'],
                         gamma=inner_clf.best_params_['gamma'],
                         kernel=inner_clf.best_params_['kernel'])
        outer_clf = classifier.fit(DFC_train_outer, label_train_outer)
        
        # Validate the classifier
        score = outer_clf.score(DFC_test_outer, label_test_outer)
        clf_score.append(score)
    
    # find the best hyper-combo
    best_idx = clf_score.index(max(clf_score))
    
    C_best_DFC.append(C_best[best_idx])
    gamma_best_DFC.append(gamma_best[best_idx])
    kernel_best_DFC.append(kernel_best[best_idx])
    print ('Best cost value:', C_best_DFC[c])
    print ('Best gamma:', gamma_best_DFC[c])
    print ('Best kernel:', kernel_best_DFC[c])
    
    classifier = SVC(class_weight='balanced',C=C_best[best_idx],gamma=gamma_best[best_idx],kernel=kernel_best[best_idx])
    final_clf = classifier.fit(LakeDFC_train, label_train)
    
    # Test the classifier
    clf_pred_DFC[:,c] = final_clf.predict(LakeDFC_test)
    clf_score_DFC.append(f1_score(label_test, clf_pred_DFC[:,c], average='macro'))
    print(str(c)+' f-1 score for Lake DFC is: '+str(clf_score_DFC[c]))
    clf_recall_DFC[c,:] = recall_score(label_test, clf_pred_DFC[:,c], average=None)

    
    ### Lake VIS ###
    # Step2: Nested 3-fold cross-validation for hyperparameter tunning
    # Outter loop: test the classifier performance
    kf_outer = StratifiedKFold(n_splits=3)
    clf_score = []
    C_best = []
    gamma_best = []
    kernel_best = []
    for train_outer, test_outer in kf_outer.split(LakeVIS_train, label_train):
        # Pull out the sample data
        label_train_outer = label_train[train_outer]
        label_test_outer = label_train[test_outer]
    
        VIS_train_outer = LakeVIS_train[train_outer,:]
        VIS_test_outer = LakeVIS_train[test_outer,:]

        ###### Inner loop: tune the hyper-parameter ######
        kf_inner = StratifiedKFold(n_splits=3)
    
        # Search over different regularization parameters: smaller values specify stronger regularization.
        parameters = {'C':[0.1,10,100],
                      'gamma':[0.001, 1, 1000],
                      'kernel':['poly', 'rbf']}
    
        inner_clf = GridSearchCV(
            SVC(class_weight='balanced'),
            parameters,
            cv=kf_inner,
            scoring='f1_macro',
            return_train_score=True)
        inner_clf.fit(VIS_train_outer, label_train_outer)
        ###### Inner loop: tune the hyper-parameter ######
    
        # Find the best hyperparameter
        C_best.append(inner_clf.best_params_['C'])
        gamma_best.append(inner_clf.best_params_['gamma'])
        kernel_best.append(inner_clf.best_params_['kernel'])
    
        # Train the classifier with the best hyperparameter using training and validation set
        classifier = SVC(class_weight='balanced',
                         C=inner_clf.best_params_['C'],
                         gamma=inner_clf.best_params_['gamma'],
                         kernel=inner_clf.best_params_['kernel'])
        outer_clf = classifier.fit(VIS_train_outer, label_train_outer)
        
        # Validate the classifier
        score = outer_clf.score(VIS_test_outer, label_test_outer)
        clf_score.append(score)
    
    # find the best hyper-combo
    best_idx = clf_score.index(max(clf_score))
    
    C_best_VIS.append(C_best[best_idx])
    gamma_best_VIS.append(gamma_best[best_idx])
    kernel_best_VIS.append(kernel_best[best_idx])
    print ('Best cost value:', C_best_VIS[c])
    print ('Best gamma:', gamma_best_VIS[c])
    print ('Best kernel:', kernel_best_VIS[c])
    
    classifier = SVC(class_weight='balanced',C=C_best[best_idx],gamma=gamma_best[best_idx],kernel=kernel_best[best_idx])
    final_clf = classifier.fit(LakeVIS_train, label_train)
    
    # Test the classifier
    clf_pred_VIS[:,c] = final_clf.predict(LakeVIS_test)
    clf_score_VIS.append(f1_score(label_test, clf_pred_VIS[:,c], average='macro'))
    print(str(c)+' f-1 score for Lake VIS is: '+str(clf_score_VIS[c]))
    clf_recall_VIS[c,:] = recall_score(label_test, clf_pred_VIS[:,c], average=None)

Best cost value: 10
Best gamma: 1
Best kernel: rbf
0 f-1 score for Lake DFC is: 0.3887757342726287
Best cost value: 100
Best gamma: 1
Best kernel: rbf
0 f-1 score for Lake VIS is: 0.4384756506415493
Best cost value: 100
Best gamma: 1
Best kernel: rbf
1 f-1 score for Lake DFC is: 0.36597069942017674
Best cost value: 100
Best gamma: 1
Best kernel: poly
1 f-1 score for Lake VIS is: 0.3317965524838337
Best cost value: 0.1
Best gamma: 1000
Best kernel: poly
2 f-1 score for Lake DFC is: 0.2881735773040121
Best cost value: 10
Best gamma: 1
Best kernel: rbf
2 f-1 score for Lake VIS is: 0.36876505615962957
Best cost value: 0.1
Best gamma: 1000
Best kernel: poly
3 f-1 score for Lake DFC is: 0.4626171748228661
Best cost value: 10
Best gamma: 1
Best kernel: rbf
3 f-1 score for Lake VIS is: 0.4436088875496217
Best cost value: 100
Best gamma: 1
Best kernel: rbf
4 f-1 score for Lake DFC is: 0.4037690632733226
Best cost value: 100
Best gamma: 1
Best kernel: poly
4 f-1 score for Lake VIS is: 0.46211645

Best cost value: 10
Best gamma: 1
Best kernel: rbf
40 f-1 score for Lake VIS is: 0.4342082700782937
Best cost value: 100
Best gamma: 1
Best kernel: poly
41 f-1 score for Lake DFC is: 0.3685074801309023
Best cost value: 10
Best gamma: 1
Best kernel: rbf
41 f-1 score for Lake VIS is: 0.4122071295003626
Best cost value: 100
Best gamma: 1
Best kernel: rbf
42 f-1 score for Lake DFC is: 0.35348702704883855
Best cost value: 10
Best gamma: 1
Best kernel: rbf
42 f-1 score for Lake VIS is: 0.45135097101041366
Best cost value: 0.1
Best gamma: 1000
Best kernel: poly
43 f-1 score for Lake DFC is: 0.2965710130465065
Best cost value: 100
Best gamma: 1
Best kernel: poly
43 f-1 score for Lake VIS is: 0.3850815167504634
Best cost value: 0.1
Best gamma: 1000
Best kernel: poly
44 f-1 score for Lake DFC is: 0.3283288967761018
Best cost value: 10
Best gamma: 1
Best kernel: rbf
44 f-1 score for Lake VIS is: 0.3787317002703263
Best cost value: 100
Best gamma: 1
Best kernel: rbf
45 f-1 score for Lake DFC is: 0

Best cost value: 100
Best gamma: 1
Best kernel: poly
81 f-1 score for Lake DFC is: 0.30120013536162604
Best cost value: 10
Best gamma: 1
Best kernel: rbf
81 f-1 score for Lake VIS is: 0.3294262600415445
Best cost value: 10
Best gamma: 1
Best kernel: rbf
82 f-1 score for Lake DFC is: 0.41522262051673825
Best cost value: 10
Best gamma: 1
Best kernel: rbf
82 f-1 score for Lake VIS is: 0.4341712680422359
Best cost value: 0.1
Best gamma: 1000
Best kernel: poly
83 f-1 score for Lake DFC is: 0.31711299904577217
Best cost value: 10
Best gamma: 1
Best kernel: rbf
83 f-1 score for Lake VIS is: 0.35845265845265845
Best cost value: 0.1
Best gamma: 1000
Best kernel: poly
84 f-1 score for Lake DFC is: 0.3521188865555415
Best cost value: 100
Best gamma: 1
Best kernel: rbf
84 f-1 score for Lake VIS is: 0.39326705040990756
Best cost value: 10
Best gamma: 1
Best kernel: rbf
85 f-1 score for Lake DFC is: 0.36863702002540144
Best cost value: 100
Best gamma: 1
Best kernel: poly
85 f-1 score for Lake VIS is

Best cost value: 100
Best gamma: 1
Best kernel: poly
121 f-1 score for Lake VIS is: 0.41935865510299486
Best cost value: 0.1
Best gamma: 1000
Best kernel: poly
122 f-1 score for Lake DFC is: 0.36080590940829776
Best cost value: 10
Best gamma: 1
Best kernel: rbf
122 f-1 score for Lake VIS is: 0.4120203949254755
Best cost value: 100
Best gamma: 1
Best kernel: rbf
123 f-1 score for Lake DFC is: 0.37619229835868495
Best cost value: 10
Best gamma: 1
Best kernel: rbf
123 f-1 score for Lake VIS is: 0.4463503837507633
Best cost value: 0.1
Best gamma: 1000
Best kernel: poly
124 f-1 score for Lake DFC is: 0.38449420949420954
Best cost value: 100
Best gamma: 1
Best kernel: rbf
124 f-1 score for Lake VIS is: 0.3307604960703224
Best cost value: 100
Best gamma: 1
Best kernel: rbf
125 f-1 score for Lake DFC is: 0.3219451838706497
Best cost value: 10
Best gamma: 1
Best kernel: rbf
125 f-1 score for Lake VIS is: 0.39534977100194496
Best cost value: 100
Best gamma: 1
Best kernel: rbf
126 f-1 score for L

Best cost value: 10
Best gamma: 1
Best kernel: rbf
161 f-1 score for Lake VIS is: 0.4868765571482006
Best cost value: 10
Best gamma: 1
Best kernel: rbf
162 f-1 score for Lake DFC is: 0.33757237620223063
Best cost value: 100
Best gamma: 1
Best kernel: rbf
162 f-1 score for Lake VIS is: 0.3728501948634772
Best cost value: 0.1
Best gamma: 1000
Best kernel: poly
163 f-1 score for Lake DFC is: 0.3767345121117277
Best cost value: 10
Best gamma: 1
Best kernel: rbf
163 f-1 score for Lake VIS is: 0.4298053137338851
Best cost value: 10
Best gamma: 1
Best kernel: rbf
164 f-1 score for Lake DFC is: 0.3376496731031648
Best cost value: 10
Best gamma: 1
Best kernel: rbf
164 f-1 score for Lake VIS is: 0.3515693327354317
Best cost value: 100
Best gamma: 1
Best kernel: rbf
165 f-1 score for Lake DFC is: 0.38421277106783985
Best cost value: 100
Best gamma: 1
Best kernel: rbf
165 f-1 score for Lake VIS is: 0.44188505183579074
Best cost value: 10
Best gamma: 1
Best kernel: rbf
166 f-1 score for Lake DFC is

...

Best cost value: 10
Best gamma: 1
Best kernel: rbf
802 f-1 score for Lake VIS is: 0.3742424541182304
Best cost value: 100
Best gamma: 1
Best kernel: rbf
803 f-1 score for Lake DFC is: 0.3736532450818165
Best cost value: 10
Best gamma: 1
Best kernel: rbf
803 f-1 score for Lake VIS is: 0.3813399299113585
Best cost value: 100
Best gamma: 1
Best kernel: rbf
804 f-1 score for Lake DFC is: 0.3151213690439804
Best cost value: 100
Best gamma: 1
Best kernel: poly
804 f-1 score for Lake VIS is: 0.40703930472809197
Best cost value: 100
Best gamma: 1
Best kernel: rbf
805 f-1 score for Lake DFC is: 0.3364427653733753
Best cost value: 10
Best gamma: 1
Best kernel: rbf
805 f-1 score for Lake VIS is: 0.36350173921753987
Best cost value: 100
Best gamma: 1
Best kernel: poly
806 f-1 score for Lake DFC is: 0.37754339898254674
Best cost value: 10
Best gamma: 1
Best kernel: rbf
806 f-1 score for Lake VIS is: 0.4166271872733767
Best cost value: 100
Best gamma: 1
Best kernel: rbf
807 f-1 score for Lake DFC is

Best cost value: 10
Best gamma: 1
Best kernel: rbf
842 f-1 score for Lake VIS is: 0.49743203514468115
Best cost value: 100
Best gamma: 1
Best kernel: rbf
843 f-1 score for Lake DFC is: 0.34904196828427886
Best cost value: 10
Best gamma: 1
Best kernel: rbf
843 f-1 score for Lake VIS is: 0.3933576904041457
Best cost value: 100
Best gamma: 1
Best kernel: rbf
844 f-1 score for Lake DFC is: 0.3781236652665224
Best cost value: 10
Best gamma: 1
Best kernel: rbf
844 f-1 score for Lake VIS is: 0.42603520391538824
Best cost value: 10
Best gamma: 1
Best kernel: rbf
845 f-1 score for Lake DFC is: 0.3228169795759647
Best cost value: 10
Best gamma: 1
Best kernel: rbf
845 f-1 score for Lake VIS is: 0.39799686573880116
Best cost value: 0.1
Best gamma: 1000
Best kernel: poly
846 f-1 score for Lake DFC is: 0.3195843207957894
Best cost value: 0.1
Best gamma: 1000
Best kernel: poly
846 f-1 score for Lake VIS is: 0.38638794206900523
Best cost value: 100
Best gamma: 1
Best kernel: rbf
847 f-1 score for Lake

Best cost value: 10
Best gamma: 1
Best kernel: rbf
882 f-1 score for Lake VIS is: 0.32446228719520637
Best cost value: 0.1
Best gamma: 1000
Best kernel: poly
883 f-1 score for Lake DFC is: 0.3588137750140317
Best cost value: 10
Best gamma: 1
Best kernel: rbf
883 f-1 score for Lake VIS is: 0.45659349093999557
Best cost value: 10
Best gamma: 1
Best kernel: rbf
884 f-1 score for Lake DFC is: 0.3315126428439122
Best cost value: 10
Best gamma: 1
Best kernel: rbf
884 f-1 score for Lake VIS is: 0.40929653570815877
Best cost value: 100
Best gamma: 1
Best kernel: rbf
885 f-1 score for Lake DFC is: 0.36686499072253825
Best cost value: 10
Best gamma: 1
Best kernel: rbf
885 f-1 score for Lake VIS is: 0.3927318295739348
Best cost value: 100
Best gamma: 1
Best kernel: rbf
886 f-1 score for Lake DFC is: 0.3618584719148629
Best cost value: 100
Best gamma: 1
Best kernel: rbf
886 f-1 score for Lake VIS is: 0.366612910091171
Best cost value: 100
Best gamma: 1
Best kernel: rbf
887 f-1 score for Lake DFC i

Best cost value: 100
Best gamma: 1
Best kernel: rbf
922 f-1 score for Lake VIS is: 0.3573294750803741
Best cost value: 100
Best gamma: 1
Best kernel: rbf
923 f-1 score for Lake DFC is: 0.4120185511162955
Best cost value: 0.1
Best gamma: 1000
Best kernel: poly
923 f-1 score for Lake VIS is: 0.29775261950165927
Best cost value: 100
Best gamma: 1
Best kernel: poly
924 f-1 score for Lake DFC is: 0.3294653758939473
Best cost value: 100
Best gamma: 1
Best kernel: poly
924 f-1 score for Lake VIS is: 0.41422590152984007
Best cost value: 10
Best gamma: 1
Best kernel: rbf
925 f-1 score for Lake DFC is: 0.34293184456523834
Best cost value: 10
Best gamma: 1
Best kernel: rbf
925 f-1 score for Lake VIS is: 0.3641819016117754
Best cost value: 100
Best gamma: 1
Best kernel: poly
926 f-1 score for Lake DFC is: 0.3922331323647113
Best cost value: 100
Best gamma: 1
Best kernel: rbf
926 f-1 score for Lake VIS is: 0.35202888172812996
Best cost value: 0.1
Best gamma: 1000
Best kernel: poly
927 f-1 score for

Best cost value: 10
Best gamma: 1
Best kernel: rbf
962 f-1 score for Lake VIS is: 0.43647801642651196
Best cost value: 100
Best gamma: 1
Best kernel: rbf
963 f-1 score for Lake DFC is: 0.4159615348520896
Best cost value: 10
Best gamma: 1
Best kernel: rbf
963 f-1 score for Lake VIS is: 0.45077555773006867
Best cost value: 100
Best gamma: 1
Best kernel: poly
964 f-1 score for Lake DFC is: 0.37306656584852066
Best cost value: 10
Best gamma: 1
Best kernel: rbf
964 f-1 score for Lake VIS is: 0.47590146980390885
Best cost value: 0.1
Best gamma: 1000
Best kernel: poly
965 f-1 score for Lake DFC is: 0.42656168954729995
Best cost value: 100
Best gamma: 1
Best kernel: poly
965 f-1 score for Lake VIS is: 0.3826089715497952
Best cost value: 100
Best gamma: 1
Best kernel: rbf
966 f-1 score for Lake DFC is: 0.3330262216106136
Best cost value: 100
Best gamma: 1
Best kernel: poly
966 f-1 score for Lake VIS is: 0.46319143819143827
Best cost value: 10
Best gamma: 1
Best kernel: rbf
967 f-1 score for Lak

In [134]:
# save the results
with open(net_enrich_dir+'/1000ClassifiersBalanced_Full.pkl', 'wb') as f:
    pickle.dump([clf_test, sample_test,
                 clf_pred_DFC, clf_score_DFC, clf_recall_DFC, C_best_DFC, gamma_best_DFC, kernel_best_DFC,
                 clf_pred_VIS, clf_score_VIS, clf_recall_VIS, C_best_VIS, gamma_best_VIS, kernel_best_VIS], f)

# load the results
#clf_test, sample_test, clf_pred_DFC, clf_score_DFC, clf_recall_DFC, C_best_DFC, gamma_best_DFC, kernel_best_DFC, clf_pred_VIS, clf_score_VIS, clf_recall_VIS, C_best_VIS, gamma_best_VIS, kernel_best_VIS = pickle.load(open(net_enrich_dir+'/1000Classifiers_Full.pkl',"rb"))


#### 1000 nulls
- In the comment `# Step0: Permutate the labels`, is the way to generate null data by permutating the network lables totally at random.
- The uncommented `# Step0: Get the spin-tested label`, is the way to generate null data by spinning the network lables controllig for spatial autocorrelation among parcels.

In the paper, we did both to generate p-values at different stringent level.

In [162]:
clf_num = 1000
parcel_num = 339
test_parcel_num = 102
net_num = 7

# Step0: Permutate the labels
#schaefer400_7Networks_perm = np.random.permutation(schaefer400_7Networks_mat)

null_test = np.zeros((test_parcel_num,clf_num))
sample_test = np.zeros((test_parcel_num,clf_num))

null_pred_DFC = np.zeros((test_parcel_num,clf_num))
null_score_DFC = []
null_recall_DFC = np.zeros((clf_num,net_num))

null_pred_VIS = np.zeros((test_parcel_num,clf_num))
null_score_VIS = []
null_recall_VIS = np.zeros((clf_num,net_num))
for c in range(clf_num):
    # Step0: Get the spin-tested label
    schaefer400_7Networks_perm = Null_7NetLabel_mat[:,c]
    
    # Step1: Split 70% for train and 30% for test
    train, test = train_test_split(list(range(339)), test_size=0.3, stratify=schaefer400_7Networks_perm)
    label_train = schaefer400_7Networks_perm[train]
    label_test = schaefer400_7Networks_perm[test]
    # save the permutated lable
    #null_test[:,c] = label_test[:,0]
    null_test[:,c] = label_test
    # save the ID of the tested parcels
    sample_test[:,c] = test

    LakeDFC_train = np.transpose(LakeDFC_schaefer400_mat[:,train])
    LakeDFC_test = np.transpose(LakeDFC_schaefer400_mat[:,test])

    LakeVIS_train = np.transpose(LakeVIS_schaefer400_mat[:,train])
    LakeVIS_test = np.transpose(LakeVIS_schaefer400_mat[:,test])
    
    
    ### Lake DFC ###
    # Step2: Random draw a hyper-combo then train the classifer
    # find the best hyper-combo
    best_idx = np.random.randint(1000, size=1)[0]
    
    
    # Step3: Train the classifier
    classifier = SVC(class_weight='balanced',C=C_best_DFC[best_idx],gamma=gamma_best_DFC[best_idx],kernel=kernel_best_DFC[best_idx])
    final_clf = classifier.fit(LakeDFC_train, label_train)
    
    # Test the classifier
    null_pred_DFC[:,c] = final_clf.predict(LakeDFC_test)
    null_score_DFC.append(f1_score(label_test, null_pred_DFC[:,c], average='macro'))
    print(str(c)+' f-1 score for Lake DFC is: '+str(null_score_DFC[c]))
    null_recall_DFC[c,:] = recall_score(label_test, null_pred_DFC[:,c], average=None)

    
    ### Lake VIS ###
    # Step2: Random draw a hyper-combo then train the classifer
    # find the best hyper-combo
    best_idx = np.random.randint(1000, size=1)[0]
    
    
    # Step3: Train the classifier
    classifier = SVC(class_weight='balanced',C=C_best_VIS[best_idx],gamma=gamma_best_VIS[best_idx],kernel=kernel_best_VIS[best_idx])
    final_clf = classifier.fit(LakeVIS_train, label_train)
    
    # Test the classifier
    null_pred_VIS[:,c] = final_clf.predict(LakeVIS_test)
    null_score_VIS.append(f1_score(label_test, null_pred_VIS[:,c], average='macro'))
    print(str(c)+' f-1 score for Lake VIS is: '+str(null_score_VIS[c]))
    null_recall_VIS[c,:] = recall_score(label_test, null_pred_VIS[:,c], average=None)

0 f-1 score for Lake DFC is: 0.2731367155012475
0 f-1 score for Lake VIS is: 0.24570245489500148
1 f-1 score for Lake DFC is: 0.3170501973332579
1 f-1 score for Lake VIS is: 0.2430340074665243
2 f-1 score for Lake DFC is: 0.23457651680651903
2 f-1 score for Lake VIS is: 0.17694676013641533
3 f-1 score for Lake DFC is: 0.20476411343758286
3 f-1 score for Lake VIS is: 0.22791579230025577
4 f-1 score for Lake DFC is: 0.31999409900937914
4 f-1 score for Lake VIS is: 0.2360912034825078
5 f-1 score for Lake DFC is: 0.1592994496793984
5 f-1 score for Lake VIS is: 0.22196153577692979
6 f-1 score for Lake DFC is: 0.25410570740582605
6 f-1 score for Lake VIS is: 0.21041525553909454
7 f-1 score for Lake DFC is: 0.16770137508018632
7 f-1 score for Lake VIS is: 0.13618413062239534
8 f-1 score for Lake DFC is: 0.3243570404000642
8 f-1 score for Lake VIS is: 0.27587479496818945
9 f-1 score for Lake DFC is: 0.21756311701468714
9 f-1 score for Lake VIS is: 0.2384476417577463
10 f-1 score for Lake DFC i

84 f-1 score for Lake VIS is: 0.23355233726119273
85 f-1 score for Lake DFC is: 0.24733500951868156
85 f-1 score for Lake VIS is: 0.32342951027161554
86 f-1 score for Lake DFC is: 0.2556654499527849
86 f-1 score for Lake VIS is: 0.223560309274595
87 f-1 score for Lake DFC is: 0.24840688332553168
87 f-1 score for Lake VIS is: 0.21574273758446647
88 f-1 score for Lake DFC is: 0.28499589988796875
88 f-1 score for Lake VIS is: 0.2377461040251738
89 f-1 score for Lake DFC is: 0.23415343915343914
89 f-1 score for Lake VIS is: 0.2887623942523517
90 f-1 score for Lake DFC is: 0.23461524672704795
90 f-1 score for Lake VIS is: 0.29183573840774646
91 f-1 score for Lake DFC is: 0.17926254559794932
91 f-1 score for Lake VIS is: 0.2564446831364125
92 f-1 score for Lake DFC is: 0.20002526862190048
92 f-1 score for Lake VIS is: 0.18128853451665505
93 f-1 score for Lake DFC is: 0.18767073496065095
93 f-1 score for Lake VIS is: 0.2801006876180424
94 f-1 score for Lake DFC is: 0.22999791009521217
94 f-1 

169 f-1 score for Lake DFC is: 0.22330303850674543
169 f-1 score for Lake VIS is: 0.27396565875828555
170 f-1 score for Lake DFC is: 0.2760875735171941
170 f-1 score for Lake VIS is: 0.2172570210117144
171 f-1 score for Lake DFC is: 0.37479922038745567
171 f-1 score for Lake VIS is: 0.3894387626486065
172 f-1 score for Lake DFC is: 0.2688285970322584
172 f-1 score for Lake VIS is: 0.28210381009871904
173 f-1 score for Lake DFC is: 0.21232069517783803
173 f-1 score for Lake VIS is: 0.21234598734598734
174 f-1 score for Lake DFC is: 0.2082528455526167
174 f-1 score for Lake VIS is: 0.2323713323713324
175 f-1 score for Lake DFC is: 0.33318552397659895
175 f-1 score for Lake VIS is: 0.3327084952084952
176 f-1 score for Lake DFC is: 0.32575019683920975
176 f-1 score for Lake VIS is: 0.2214895830309364
177 f-1 score for Lake DFC is: 0.20016686590069435
177 f-1 score for Lake VIS is: 0.2885871892083073
178 f-1 score for Lake DFC is: 0.2856341912280406
178 f-1 score for Lake VIS is: 0.24465579

252 f-1 score for Lake DFC is: 0.2463453501015593
252 f-1 score for Lake VIS is: 0.2715405284370802
253 f-1 score for Lake DFC is: 0.25001850646449947
253 f-1 score for Lake VIS is: 0.29851988736557905
254 f-1 score for Lake DFC is: 0.23683269370850393
254 f-1 score for Lake VIS is: 0.1419304032085987
255 f-1 score for Lake DFC is: 0.18939212387488252
255 f-1 score for Lake VIS is: 0.23160489171882723
256 f-1 score for Lake DFC is: 0.24063066477813022
256 f-1 score for Lake VIS is: 0.2555587380050922
257 f-1 score for Lake DFC is: 0.3248007809098034
257 f-1 score for Lake VIS is: 0.2504764658899997
258 f-1 score for Lake DFC is: 0.2437064884433305
258 f-1 score for Lake VIS is: 0.21187779140119667
259 f-1 score for Lake DFC is: 0.16704817884219877
259 f-1 score for Lake VIS is: 0.2766000492729265
260 f-1 score for Lake DFC is: 0.2622138238885471
260 f-1 score for Lake VIS is: 0.2507355115959427
261 f-1 score for Lake DFC is: 0.23366997148509755
261 f-1 score for Lake VIS is: 0.19431184

335 f-1 score for Lake DFC is: 0.24128390805736347
335 f-1 score for Lake VIS is: 0.22321943697875918
336 f-1 score for Lake DFC is: 0.21932931606939143
336 f-1 score for Lake VIS is: 0.288016514464911
337 f-1 score for Lake DFC is: 0.2602745293396948
337 f-1 score for Lake VIS is: 0.24915094044070488
338 f-1 score for Lake DFC is: 0.2178269525446201
338 f-1 score for Lake VIS is: 0.1991697609810213
339 f-1 score for Lake DFC is: 0.19823602572205232
339 f-1 score for Lake VIS is: 0.2360070348850152
340 f-1 score for Lake DFC is: 0.28109251386027634
340 f-1 score for Lake VIS is: 0.3001767214062659
341 f-1 score for Lake DFC is: 0.2960582762737502
341 f-1 score for Lake VIS is: 0.27053364006924374
342 f-1 score for Lake DFC is: 0.20395159088557208
342 f-1 score for Lake VIS is: 0.1992352672294214
343 f-1 score for Lake DFC is: 0.2802033805793205
343 f-1 score for Lake VIS is: 0.33076810303701054
344 f-1 score for Lake DFC is: 0.25841571402547014
344 f-1 score for Lake VIS is: 0.27197862

...

585 f-1 score for Lake VIS is: 0.26252960008258436
586 f-1 score for Lake DFC is: 0.3131566729436083
586 f-1 score for Lake VIS is: 0.2948646125116713
587 f-1 score for Lake DFC is: 0.33714994053861574
587 f-1 score for Lake VIS is: 0.3419552704024754
588 f-1 score for Lake DFC is: 0.22607190018055895
588 f-1 score for Lake VIS is: 0.33885789807933264
589 f-1 score for Lake DFC is: 0.26798213960820944
589 f-1 score for Lake VIS is: 0.23231619016889646
590 f-1 score for Lake DFC is: 0.17666313554815294
590 f-1 score for Lake VIS is: 0.15881563268322849
591 f-1 score for Lake DFC is: 0.24700356333009393
591 f-1 score for Lake VIS is: 0.3230867464009502
592 f-1 score for Lake DFC is: 0.2697833658114796
592 f-1 score for Lake VIS is: 0.26250345687349486
593 f-1 score for Lake DFC is: 0.2121657019489837
593 f-1 score for Lake VIS is: 0.22880047744497456
594 f-1 score for Lake DFC is: 0.21288903187749678
594 f-1 score for Lake VIS is: 0.2430956998183889
595 f-1 score for Lake DFC is: 0.32688

668 f-1 score for Lake DFC is: 0.22912773501008796
668 f-1 score for Lake VIS is: 0.24693239397072153
669 f-1 score for Lake DFC is: 0.28498834498834497
669 f-1 score for Lake VIS is: 0.2041517527490715
670 f-1 score for Lake DFC is: 0.21747860434496377
670 f-1 score for Lake VIS is: 0.23642769928484214
671 f-1 score for Lake DFC is: 0.2085034013605442
671 f-1 score for Lake VIS is: 0.22193678204567496
672 f-1 score for Lake DFC is: 0.21504294595160786
672 f-1 score for Lake VIS is: 0.3143983117722614
673 f-1 score for Lake DFC is: 0.31928520396012655
673 f-1 score for Lake VIS is: 0.30427320353428733
674 f-1 score for Lake DFC is: 0.24111435745775392
674 f-1 score for Lake VIS is: 0.24051463728590364
675 f-1 score for Lake DFC is: 0.19703557485949513
675 f-1 score for Lake VIS is: 0.24281224372757554
676 f-1 score for Lake DFC is: 0.20046027865986502
676 f-1 score for Lake VIS is: 0.2633235624873255
677 f-1 score for Lake DFC is: 0.2347577189192096
677 f-1 score for Lake VIS is: 0.221

753 f-1 score for Lake VIS is: 0.19495706083851982
754 f-1 score for Lake DFC is: 0.14854497354497356
754 f-1 score for Lake VIS is: 0.19315036359371335
755 f-1 score for Lake DFC is: 0.3006628769124263
755 f-1 score for Lake VIS is: 0.2851195397377663
756 f-1 score for Lake DFC is: 0.26969960921573827
756 f-1 score for Lake VIS is: 0.31286813996006335
757 f-1 score for Lake DFC is: 0.32523078401321
757 f-1 score for Lake VIS is: 0.2647541055435792
758 f-1 score for Lake DFC is: 0.1840271410232899
758 f-1 score for Lake VIS is: 0.20802378245727998
759 f-1 score for Lake DFC is: 0.2807999309660439
759 f-1 score for Lake VIS is: 0.32327550689827467
760 f-1 score for Lake DFC is: 0.2285460667376022
760 f-1 score for Lake VIS is: 0.30609786344868734
761 f-1 score for Lake DFC is: 0.2892702479659001
761 f-1 score for Lake VIS is: 0.21424143108353635
762 f-1 score for Lake DFC is: 0.20181821301863692
762 f-1 score for Lake VIS is: 0.17662337662337663
763 f-1 score for Lake DFC is: 0.30927750

837 f-1 score for Lake DFC is: 0.26117503678479287
837 f-1 score for Lake VIS is: 0.2873369475488584
838 f-1 score for Lake DFC is: 0.23873879943172652
838 f-1 score for Lake VIS is: 0.2434283443600214
839 f-1 score for Lake DFC is: 0.2029499445465832
839 f-1 score for Lake VIS is: 0.25623067408781697
840 f-1 score for Lake DFC is: 0.21559829250271134
840 f-1 score for Lake VIS is: 0.2192450029461914
841 f-1 score for Lake DFC is: 0.31150186076294445
841 f-1 score for Lake VIS is: 0.2722039864897008
842 f-1 score for Lake DFC is: 0.26356251676891373
842 f-1 score for Lake VIS is: 0.28374605344530157
843 f-1 score for Lake DFC is: 0.2229878314759972
843 f-1 score for Lake VIS is: 0.17229592829079346
844 f-1 score for Lake DFC is: 0.32179342425887775
844 f-1 score for Lake VIS is: 0.3468989397560826
845 f-1 score for Lake DFC is: 0.3050592227062815
845 f-1 score for Lake VIS is: 0.26115609880315765
846 f-1 score for Lake DFC is: 0.19052607539449642
846 f-1 score for Lake VIS is: 0.187580

920 f-1 score for Lake DFC is: 0.2916998730227786
920 f-1 score for Lake VIS is: 0.2351556490957413
921 f-1 score for Lake DFC is: 0.22464650195742633
921 f-1 score for Lake VIS is: 0.3797839452702417
922 f-1 score for Lake DFC is: 0.3867462293172232
922 f-1 score for Lake VIS is: 0.3184572873934576
923 f-1 score for Lake DFC is: 0.3397670839145493
923 f-1 score for Lake VIS is: 0.27386685668419414
924 f-1 score for Lake DFC is: 0.23155031243266538
924 f-1 score for Lake VIS is: 0.20523061902372247
925 f-1 score for Lake DFC is: 0.23621550162903546
925 f-1 score for Lake VIS is: 0.2736184065834278
926 f-1 score for Lake DFC is: 0.2332742463119885
926 f-1 score for Lake VIS is: 0.2589573715194153
927 f-1 score for Lake DFC is: 0.32425921001210184
927 f-1 score for Lake VIS is: 0.3067564210238671
928 f-1 score for Lake DFC is: 0.20263022910074355
928 f-1 score for Lake VIS is: 0.19567502375384146
929 f-1 score for Lake DFC is: 0.20643520361565473
929 f-1 score for Lake VIS is: 0.27649361

In [163]:
# save the results
with open(net_enrich_dir+'/1000NullsBalancedSpin_Full.pkl', 'wb') as f:
    pickle.dump([null_test, sample_test,
                 null_pred_DFC, null_score_DFC, null_recall_DFC,
                 null_pred_VIS, null_score_VIS, null_recall_VIS], f)

# load the results
#null_test, sample_test, null_pred_DFC, null_score_DFC, null_recall_DFC, null_pred_VIS, null_score_VIS, null_recall_VIS = pickle.load(open(net_enrich_dir+'/1000Nulls_Full.pkl',"rb"))


## PartB: Ensemble Lake DFC and Lake VIS
Here we ensemble classifiers trained on Lake DFC and Lake VIS by averaging their one-vs-one decision functions on each network the parcel may belong to, so if the mean is above 0, the ensembled vote goes to the first network being compared, and if the mean is below 0, the ensembled vote goest to the other network being compared.

Below is the example structure of a one-vs-one vote for a single classifier deciding which network a parcel belongs to. Among each 1 of the 21 comparisons, the `pos` vector encode the first network in the comparison pair, and the `neg` vector encode the second network in the comparison pair. `Decision function` contains the confidence of the current vote going to the first or second network. After all the 21 comparisons are done, the classifier will pick whichever network got the max votes. Please find more details is in the [Sklearn SVM document](https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsOneClassifier.html#sklearn.multiclass.OneVsOneClassifier)

In [3]:
pos = np.asarray([1,1,1,1,1,1,2,2,2,2,2,3,3,3,3,4,4,4,5,5,6])
neg = np.asarray([2,3,4,5,6,7,3,4,5,6,7,4,5,6,7,5,6,7,6,7,7])
votes = np.stack((pos,neg),axis=1)
votes

array([[1, 2],
       [1, 3],
       [1, 4],
       [1, 5],
       [1, 6],
       [1, 7],
       [2, 3],
       [2, 4],
       [2, 5],
       [2, 6],
       [2, 7],
       [3, 4],
       [3, 5],
       [3, 6],
       [3, 7],
       [4, 5],
       [4, 6],
       [4, 7],
       [5, 6],
       [5, 7],
       [6, 7]])

### Part B1: Left Hemisphere Only
#### 1000 Classifiers

In [17]:
clf_num = 1000
parcel_num = 191
test_parcel_num = 58
net_num = 7
schaefer400_7Networks_mat_left = schaefer400_7Networks_mat[:191,:]
LakeDFC_schaefer400_mat_left = LakeDFC_schaefer400_mat[:,:191]
LakeVIS_schaefer400_mat_left = LakeVIS_schaefer400_mat[:,:191]

clf_test = np.zeros((test_parcel_num,clf_num))
sample_test = np.zeros((test_parcel_num,clf_num))
clf_pred_Both = np.zeros((test_parcel_num,clf_num))
clf_score_Both = []
clf_recall_Both = np.zeros((clf_num,net_num))

clf_pred_DFC = np.zeros((test_parcel_num,clf_num))
clf_score_DFC = []
clf_recall_DFC = np.zeros((clf_num,net_num))
C_best_DFC = []
gamma_best_DFC = []
kernel_best_DFC = []
degree_best_DFC = []

clf_pred_VIS = np.zeros((test_parcel_num,clf_num))
clf_score_VIS = []
clf_recall_VIS = np.zeros((clf_num,net_num))
C_best_VIS = []
gamma_best_VIS = []
kernel_best_VIS = []
degree_best_VIS = []
for c in range(clf_num):
    
    # Step1: Split 70% for train and 30% for test
    train, test = train_test_split(list(range(parcel_num)), test_size=0.3, stratify=schaefer400_7Networks_mat_left,random_state=c)
    label_train = schaefer400_7Networks_mat_left[train]
    label_test = schaefer400_7Networks_mat_left[test]
    # save the real category of the tested parcels
    clf_test[:,c] = label_test[:,0]
    # save the ID of the tested parcels
    sample_test[:,c] = test

    LakeDFC_train = np.transpose(LakeDFC_schaefer400_mat_left[:,train])
    LakeDFC_test = np.transpose(LakeDFC_schaefer400_mat_left[:,test])

    LakeVIS_train = np.transpose(LakeVIS_schaefer400_mat_left[:,train])
    LakeVIS_test = np.transpose(LakeVIS_schaefer400_mat_left[:,test])
    
    
    ### Lake DFC ###
    # Step2: Nested 3-fold cross-validation for hyperparameter tunning
    # Outter loop: test the classifier performance
    kf_outer = StratifiedKFold(n_splits=3,shuffle=True,random_state=c)
    clf_score = []
    C_best = []
    gamma_best = []
    kernel_best = []
    degree_best = []
    for train_outer, test_outer in kf_outer.split(LakeDFC_train, label_train):
        # Pull out the sample data
        label_train_outer = label_train[train_outer]
        label_test_outer = label_train[test_outer]
    
        DFC_train_outer = LakeDFC_train[train_outer,:]
        DFC_test_outer = LakeDFC_train[test_outer,:]

        ###### Inner loop: tune the hyper-parameter ######
        kf_inner = StratifiedKFold(n_splits=2,shuffle=True)
    
        # Search over different regularization parameters: smaller values specify stronger regularization.
        parameters = {'C':[1e-05, 1e-03, 0.1, 10, 1e03, 1e05],
                      'gamma':[1e-03, 0.1, 10, 1e03],
                      'degree':[2,3,4],
                      'kernel':['poly', 'rbf']}
        
        inner_clf = GridSearchCV(
            SVC(class_weight='balanced',random_state=c),
            parameters,
            cv=kf_inner,
            scoring='f1_macro',
            return_train_score=True)
        inner_clf.fit(DFC_train_outer, label_train_outer)
        ###### Inner loop: tune the hyper-parameter ######
    
        # Find the best hyperparameter
        C_best.append(inner_clf.best_params_['C'])
        gamma_best.append(inner_clf.best_params_['gamma'])
        kernel_best.append(inner_clf.best_params_['kernel'])
        degree_best.append(inner_clf.best_params_['degree'])
    
        # Train the classifier with the best hyperparameter using training and validation set
        classifier = SVC(class_weight='balanced',
                         random_state=c,
                         C=inner_clf.best_params_['C'],
                         gamma=inner_clf.best_params_['gamma'],
                         degree=inner_clf.best_params_['degree'],
                         kernel=inner_clf.best_params_['kernel'])
        outer_clf = classifier.fit(DFC_train_outer, label_train_outer)
        
        # Validate the classifier
        score = outer_clf.score(DFC_test_outer, label_test_outer)
        clf_score.append(score)
    
    # find the best hyper-combo
    best_idx = clf_score.index(max(clf_score))
    
    C_best_DFC.append(C_best[best_idx])
    gamma_best_DFC.append(gamma_best[best_idx])
    kernel_best_DFC.append(kernel_best[best_idx])
    degree_best_DFC.append(degree_best[best_idx])
    print ('Best cost value:', C_best_DFC[c])
    print ('Best gamma:', gamma_best_DFC[c])
    print ('Best kernel:', kernel_best_DFC[c])
    print ('Best degree:', degree_best_DFC[c])
    
    classifier = SVC(class_weight='balanced',
                     random_state=c,
                     C=C_best[best_idx],
                     gamma=gamma_best[best_idx],
                     kernel=kernel_best[best_idx],
                     degree=degree_best[best_idx],
                     decision_function_shape='ovo')
    final_clf_DFC = classifier.fit(LakeDFC_train, label_train)
    
    # Test the classifier
    clf_pred_DFC[:,c] = final_clf_DFC.predict(LakeDFC_test)
    clf_score_DFC.append(f1_score(label_test, clf_pred_DFC[:,c], average='macro'))
    print(str(c)+' f-1 score for Lake DFC is: '+str(clf_score_DFC[c]))
    clf_recall_DFC[c,:] = recall_score(label_test, clf_pred_DFC[:,c], average=None)

    
    ### Lake VIS ###
    # Step2: Nested 3-fold cross-validation for hyperparameter tunning
    # Outter loop: test the classifier performance
    kf_outer = StratifiedKFold(n_splits=3,shuffle=True,random_state=c)
    clf_score = []
    C_best = []
    gamma_best = []
    kernel_best = []
    degree_best = []
    for train_outer, test_outer in kf_outer.split(LakeVIS_train, label_train):
        # Pull out the sample data
        label_train_outer = label_train[train_outer]
        label_test_outer = label_train[test_outer]
    
        VIS_train_outer = LakeVIS_train[train_outer,:]
        VIS_test_outer = LakeVIS_train[test_outer,:]

        ###### Inner loop: tune the hyper-parameter ######
        kf_inner = StratifiedKFold(n_splits=3,shuffle=True,random_state=c)
    
        # Search over different regularization parameters: smaller values specify stronger regularization.
        parameters = {'C':[1e-05, 1e-03, 0.1, 10, 1e03, 1e05],
                      'gamma':[1e-03, 0.1, 10, 1e03],
                      'degree':[2,3,4],
                      'kernel':['poly', 'rbf']}
    
        inner_clf = GridSearchCV(
            SVC(class_weight='balanced',random_state=c),
            parameters,
            cv=kf_inner,
            scoring='f1_macro',
            return_train_score=True)
        inner_clf.fit(VIS_train_outer, label_train_outer)
        ###### Inner loop: tune the hyper-parameter ######
    
        # Find the best hyperparameter
        C_best.append(inner_clf.best_params_['C'])
        gamma_best.append(inner_clf.best_params_['gamma'])
        kernel_best.append(inner_clf.best_params_['kernel'])
        degree_best.append(inner_clf.best_params_['degree'])
    
        # Train the classifier with the best hyperparameter using training and validation set
        classifier = SVC(class_weight='balanced',
                         random_state=c,
                         C=inner_clf.best_params_['C'],
                         gamma=inner_clf.best_params_['gamma'],
                         degree=inner_clf.best_params_['degree'],
                         kernel=inner_clf.best_params_['kernel'])
        outer_clf = classifier.fit(VIS_train_outer, label_train_outer)
        
        # Validate the classifier
        score = outer_clf.score(VIS_test_outer, label_test_outer)
        clf_score.append(score)
    
    # find the best hyper-combo
    best_idx = clf_score.index(max(clf_score))
    
    C_best_VIS.append(C_best[best_idx])
    gamma_best_VIS.append(gamma_best[best_idx])
    kernel_best_VIS.append(kernel_best[best_idx])
    degree_best_VIS.append(degree_best[best_idx])
    print ('Best cost value:', C_best_VIS[c])
    print ('Best gamma:', gamma_best_VIS[c])
    print ('Best kernel:', kernel_best_VIS[c])
    print ('Best degree:', degree_best_VIS[c])
    
    classifier = SVC(class_weight='balanced',
                     random_state=c,
                     C=C_best[best_idx],
                     gamma=gamma_best[best_idx],
                     kernel=kernel_best[best_idx],
                     degree=degree_best[best_idx],
                    decision_function_shape='ovo')
    final_clf_VIS = classifier.fit(LakeVIS_train, label_train)
    
    # Test the classifier
    clf_pred_VIS[:,c] = final_clf_VIS.predict(LakeVIS_test)
    clf_score_VIS.append(f1_score(label_test, clf_pred_VIS[:,c], average='macro'))
    print(str(c)+' f-1 score for Lake VIS is: '+str(clf_score_VIS[c]))
    clf_recall_VIS[c,:] = recall_score(label_test, clf_pred_VIS[:,c], average=None)
    
    ### Ensembling VIS and DFC ###
    decision_function_both = np.stack((final_clf_DFC.decision_function(LakeDFC_test),final_clf_VIS.decision_function(LakeVIS_test)),axis=2)
    decision_function_both = np.mean(decision_function_both,axis=2)
    decisions = np.where(decision_function_both>0,0,1) #0:positive, 1:negative
    ########### code below are for you to have fun with what's happening ###########
    #decision_function_DFC = final_clf_DFC.decision_function(LakeDFC_test)         #
    #decision_function_VIS = final_clf_VIS.decision_function(LakeVIS_test)         #
    #decisions_DFC = np.where(decision_function_DFC>0,0,1) #0:positive, 1:negative #
    #decisions_VIS = np.where(decision_function_VIS>0,0,1) #0:positive, 1:negative #
    ################################################################################
    for s in range(test_parcel_num):
        predictions = []
        # loop through all the classifiers:
        for vc in range(len(votes)):
            index = decisions[s,vc]
            #index = decisions_DFC[s,vc]
            #predictions.append(votes[vc,index])
            #index = decisions_VIS[s,vc]
            predictions.append(votes[vc,index])
        
        # find the category with most votes
        clf_pred_Both[s,c] = int(st.mode(predictions))
        
    clf_recall_Both[c,:] = recall_score(label_test[:,0], clf_pred_Both[:,c], average=None)
    clf_score_Both.append(f1_score(label_test[:,0], clf_pred_Both[:,c], average='macro'))
    print(str(c)+' f-1 score for BOTH is: '+str(clf_score_Both[c]))

Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
0 f-1 score for Lake DFC is: 0.3813797313797314
Best cost value: 0.1
Best gamma: 10
Best kernel: poly
Best degree: 3
0 f-1 score for Lake VIS is: 0.339244402214327
0 f-1 score for BOTH is: 0.4127879221864184
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
1 f-1 score for Lake DFC is: 0.4416666666666666
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
1 f-1 score for Lake VIS is: 0.3770608243297318
1 f-1 score for BOTH is: 0.36494265967950185
Best cost value: 0.1
Best gamma: 1000.0
Best kernel: poly
Best degree: 2
2 f-1 score for Lake DFC is: 0.3454435601991993
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
2 f-1 score for Lake VIS is: 0.3711193568336425
2 f-1 score for BOTH is: 0.375943104514533
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
3 f-1 score for Lake DFC is: 0.4739328413404909
Best cost value

Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
28 f-1 score for Lake VIS is: 0.439245395127748
28 f-1 score for BOTH is: 0.40118215118215117
Best cost value: 0.1
Best gamma: 10
Best kernel: poly
Best degree: 4
29 f-1 score for Lake DFC is: 0.2551448497018945
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
29 f-1 score for Lake VIS is: 0.3895661221180903
29 f-1 score for BOTH is: 0.4009286698362328
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
30 f-1 score for Lake DFC is: 0.39978354978354985
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
30 f-1 score for Lake VIS is: 0.414157333894176
30 f-1 score for BOTH is: 0.4398226335068441
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
31 f-1 score for Lake DFC is: 0.40636001769128705
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
31 f-1 score for Lake VIS is: 0.34884291098355674
31 

Best cost value: 1e-05
Best gamma: 1000.0
Best kernel: poly
Best degree: 3
57 f-1 score for Lake DFC is: 0.2613045218087235
Best cost value: 100000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
57 f-1 score for Lake VIS is: 0.36308515501062083
57 f-1 score for BOTH is: 0.33252223966509675
Best cost value: 0.001
Best gamma: 1000.0
Best kernel: poly
Best degree: 2
58 f-1 score for Lake DFC is: 0.24982345240965928
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
58 f-1 score for Lake VIS is: 0.3105539111731062
58 f-1 score for BOTH is: 0.3351639801219633
Best cost value: 0.1
Best gamma: 1000.0
Best kernel: poly
Best degree: 2
59 f-1 score for Lake DFC is: 0.4034425891568749
Best cost value: 100000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
59 f-1 score for Lake VIS is: 0.4015032586461157
59 f-1 score for BOTH is: 0.45889393257814304
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
60 f-1 score for Lake DFC is: 0.4660624249699879

Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
85 f-1 score for Lake VIS is: 0.3703601953601954
85 f-1 score for BOTH is: 0.36669531448943216
Best cost value: 10
Best gamma: 10
Best kernel: rbf
Best degree: 2
86 f-1 score for Lake DFC is: 0.3812895291725607
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
86 f-1 score for Lake VIS is: 0.3692166925249632
86 f-1 score for BOTH is: 0.3432661740932418
Best cost value: 1e-05
Best gamma: 1000.0
Best kernel: poly
Best degree: 4
87 f-1 score for Lake DFC is: 0.3784454778079814
Best cost value: 100000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
87 f-1 score for Lake VIS is: 0.4490304835188784
87 f-1 score for BOTH is: 0.3722837313062876
Best cost value: 1000.0
Best gamma: 10
Best kernel: rbf
Best degree: 2
88 f-1 score for Lake DFC is: 0.39995392096232435
Best cost value: 0.001
Best gamma: 1000.0
Best kernel: poly
Best degree: 2
88 f-1 score for Lake VIS is: 0.442072213500785
88 f-1 scor

Best cost value: 1e-05
Best gamma: 1000.0
Best kernel: poly
Best degree: 3
114 f-1 score for Lake DFC is: 0.36423021423021423
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
114 f-1 score for Lake VIS is: 0.4440596440596441
114 f-1 score for BOTH is: 0.46368446368446364
Best cost value: 1000.0
Best gamma: 10
Best kernel: rbf
Best degree: 2
115 f-1 score for Lake DFC is: 0.3082232893157263
Best cost value: 1000.0
Best gamma: 10
Best kernel: rbf
Best degree: 2
115 f-1 score for Lake VIS is: 0.32277638328058494
115 f-1 score for BOTH is: 0.37260684629105684
Best cost value: 100000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
116 f-1 score for Lake DFC is: 0.38688501041442214
Best cost value: 1000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
116 f-1 score for Lake VIS is: 0.2592433464613916
116 f-1 score for BOTH is: 0.394043717351988
Best cost value: 100000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
117 f-1 score for Lake DFC is: 0.2811688311

...

Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
871 f-1 score for Lake VIS is: 0.378031212484994
871 f-1 score for BOTH is: 0.3952942157911102
Best cost value: 1e-05
Best gamma: 1000.0
Best kernel: poly
Best degree: 4
872 f-1 score for Lake DFC is: 0.27165789392680145
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
872 f-1 score for Lake VIS is: 0.3674636474636474
872 f-1 score for BOTH is: 0.3913308913308913
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
873 f-1 score for Lake DFC is: 0.4064102564102564
Best cost value: 100000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
873 f-1 score for Lake VIS is: 0.3704821058858326
873 f-1 score for BOTH is: 0.3675491175491175
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
874 f-1 score for Lake DFC is: 0.3900087515267781
Best cost value: 10
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
874 f-1 score for Lake VIS is: 0.3419358419358419
874

Best cost value: 10
Best gamma: 10
Best kernel: rbf
Best degree: 2
900 f-1 score for Lake DFC is: 0.3288461538461539
Best cost value: 0.1
Best gamma: 10
Best kernel: poly
Best degree: 2
900 f-1 score for Lake VIS is: 0.43517248796751895
900 f-1 score for BOTH is: 0.4149082663368378
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
901 f-1 score for Lake DFC is: 0.3630385487528344
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
901 f-1 score for Lake VIS is: 0.45328023070989826
901 f-1 score for BOTH is: 0.41802648332060105
Best cost value: 100000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
902 f-1 score for Lake DFC is: 0.30203019892460886
Best cost value: 0.001
Best gamma: 1000.0
Best kernel: poly
Best degree: 2
902 f-1 score for Lake VIS is: 0.31647083075654503
902 f-1 score for BOTH is: 0.336694969869979
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
903 f-1 score for Lake DFC is: 0.4458955232410609

Best cost value: 1000.0
Best gamma: 10
Best kernel: rbf
Best degree: 2
928 f-1 score for Lake DFC is: 0.24052055604850636
Best cost value: 0.1
Best gamma: 10
Best kernel: poly
Best degree: 3
928 f-1 score for Lake VIS is: 0.2970521541950113
928 f-1 score for BOTH is: 0.30186940959436714
Best cost value: 10
Best gamma: 10
Best kernel: rbf
Best degree: 2
929 f-1 score for Lake DFC is: 0.2862558356676003
Best cost value: 10
Best gamma: 10
Best kernel: rbf
Best degree: 2
929 f-1 score for Lake VIS is: 0.4062875874987676
929 f-1 score for BOTH is: 0.44694458980173263
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
930 f-1 score for Lake DFC is: 0.3116841202884316
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
930 f-1 score for Lake VIS is: 0.31290489322525933
930 f-1 score for BOTH is: 0.3080523179171334
Best cost value: 1000.0
Best gamma: 10
Best kernel: rbf
Best degree: 2
931 f-1 score for Lake DFC is: 0.27054688231158824
Best cost valu

Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
956 f-1 score for Lake VIS is: 0.3336394276486443
956 f-1 score for BOTH is: 0.34669132037553085
Best cost value: 1e-05
Best gamma: 1000.0
Best kernel: poly
Best degree: 3
957 f-1 score for Lake DFC is: 0.31733980305408876
Best cost value: 100000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
957 f-1 score for Lake VIS is: 0.3538688202553749
957 f-1 score for BOTH is: 0.3979030868431791
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
958 f-1 score for Lake DFC is: 0.44440501953289674
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
958 f-1 score for Lake VIS is: 0.3903784124372359
958 f-1 score for BOTH is: 0.3809757236227824
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
959 f-1 score for Lake DFC is: 0.3753245991232261
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
959 f-1 score for Lake VIS is: 0.4223268794697

Best cost value: 100000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
984 f-1 score for Lake VIS is: 0.3741659773917839
984 f-1 score for BOTH is: 0.3185456000582051
Best cost value: 1e-05
Best gamma: 1000.0
Best kernel: poly
Best degree: 3
985 f-1 score for Lake DFC is: 0.36245193729665776
Best cost value: 0.1
Best gamma: 1000.0
Best kernel: poly
Best degree: 2
985 f-1 score for Lake VIS is: 0.32541851523217974
985 f-1 score for BOTH is: 0.4164428386984778
Best cost value: 1e-05
Best gamma: 1000.0
Best kernel: poly
Best degree: 3
986 f-1 score for Lake DFC is: 0.28418640183346067
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
986 f-1 score for Lake VIS is: 0.38620349325500947
986 f-1 score for BOTH is: 0.3792947063883024
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
987 f-1 score for Lake DFC is: 0.3496444032158318
Best cost value: 0.1
Best gamma: 1000.0
Best kernel: poly
Best degree: 2
987 f-1 score for Lake VIS is: 0.3179405

In [18]:
# save the result
with open(net_enrich_dir+'/1000ClassifiersBalancedEnsembeled_Left_2.pkl', 'wb') as f:
    pickle.dump([clf_test, sample_test, clf_pred_Both, clf_score_Both, clf_recall_Both,
                 clf_pred_DFC, clf_score_DFC, clf_recall_DFC, C_best_DFC, gamma_best_DFC, kernel_best_DFC,degree_best_DFC,
                 clf_pred_VIS, clf_score_VIS, clf_recall_VIS, C_best_VIS, gamma_best_VIS, kernel_best_VIS,degree_best_VIS], f)

In [19]:
# load the results
clf_test, sample_test,clf_pred_Both,clf_score_Both,clf_recall_Both, clf_pred_DFC, clf_score_DFC, clf_recall_DFC, C_best_DFC, gamma_best_DFC, kernel_best_DFC, degree_best_DFC, clf_pred_VIS, clf_score_VIS, clf_recall_VIS, C_best_VIS, gamma_best_VIS, kernel_best_VIS, degree_best_VIS = pickle.load(open(net_enrich_dir+'/1000ClassifiersBalancedEnsembeled_Left_2.pkl',"rb"))

#### 1000 permutated nulls
Null datas are generated by permutating the network labels totally at random.

In [43]:
clf_num = 1000
parcel_num = 191
test_parcel_num = 58
net_num = 7
schaefer400_7Networks_mat_left = schaefer400_7Networks_mat[:191,:]
LakeDFC_schaefer400_mat_left = LakeDFC_schaefer400_mat[:,:191]
LakeVIS_schaefer400_mat_left = LakeVIS_schaefer400_mat[:,:191]

# Step0: Permutate the labels
np.random.seed(6666)
schaefer400_7Networks_perm = np.random.permutation(schaefer400_7Networks_mat_left)

null_test = np.zeros((test_parcel_num,clf_num))
sample_test = np.zeros((test_parcel_num,clf_num))

null_pred_Both = np.zeros((test_parcel_num,clf_num))
null_score_Both = []
null_recall_Both = np.zeros((clf_num,net_num))

null_pred_DFC = np.zeros((test_parcel_num,clf_num))
null_score_DFC = []
null_recall_DFC = np.zeros((clf_num,net_num))

null_pred_VIS = np.zeros((test_parcel_num,clf_num))
null_score_VIS = []
null_recall_VIS = np.zeros((clf_num,net_num))
for c in range(clf_num):

    # Step1: Split 70% for train and 30% for test
    train, test = train_test_split(list(range(parcel_num)), test_size=0.3, stratify=schaefer400_7Networks_perm,random_state=c+1000)
    label_train = schaefer400_7Networks_perm[train]
    label_test = schaefer400_7Networks_perm[test]
    # save the permutated lable
    null_test[:,c] = label_test[:,0]
    # save the ID of the tested parcels
    sample_test[:,c] = test

    LakeDFC_train = np.transpose(LakeDFC_schaefer400_mat_left[:,train])
    LakeDFC_test = np.transpose(LakeDFC_schaefer400_mat_left[:,test])

    LakeVIS_train = np.transpose(LakeVIS_schaefer400_mat_left[:,train])
    LakeVIS_test = np.transpose(LakeVIS_schaefer400_mat_left[:,test])


    ### Lake DFC ###
    # Step2: Random draw a hyper-combo then train the classifer
    # find the best hyper-combo
    np.random.seed(c+1000)
    best_idx = np.random.randint(1000, size=1)[0]

    # Step3: Train the classifier
    classifier = SVC(class_weight='balanced',
                     random_state=c+1000,
                     C=C_best_DFC[best_idx],
                     gamma=gamma_best_DFC[best_idx],
                     degree=degree_best_DFC[best_idx],
                     kernel=kernel_best_DFC[best_idx],
                     decision_function_shape='ovo')
    final_clf_DFC = classifier.fit(LakeDFC_train, label_train)

    # Test the classifier
    null_pred_DFC[:,c] = final_clf_DFC.predict(LakeDFC_test)
    null_score_DFC.append(f1_score(label_test, null_pred_DFC[:,c], average='macro'))
    print(str(c)+' f-1 score for Lake DFC is: '+str(null_score_DFC[c]))
    null_recall_DFC[c,:] = recall_score(label_test, null_pred_DFC[:,c], average=None)


    ### Lake VIS ###
    # Step2: Random draw a hyper-combo then train the classifer
    # find the best hyper-combo
    np.random.seed(c+1000)
    best_idx = np.random.randint(1000, size=1)[0]


    # Step3: Train the classifier
    classifier = SVC(class_weight='balanced',
                     random_state=c+1000,
                     C=C_best_VIS[best_idx],
                     gamma=gamma_best_VIS[best_idx],
                     degree=degree_best_VIS[best_idx],
                     kernel=kernel_best_VIS[best_idx],
                     decision_function_shape='ovo')
    final_clf_VIS = classifier.fit(LakeVIS_train, label_train)

    # Test the classifier
    null_pred_VIS[:,c] = final_clf_VIS.predict(LakeVIS_test)
    null_score_VIS.append(f1_score(label_test, null_pred_VIS[:,c], average='macro'))
    print(str(c)+' f-1 score for Lake VIS is: '+str(null_score_VIS[c]))
    null_recall_VIS[c,:] = recall_score(label_test, null_pred_VIS[:,c], average=None)
    
    ### Ensembling VIS and DFC ###
    decision_function_both = np.stack((final_clf_DFC.decision_function(LakeDFC_test),final_clf_VIS.decision_function(LakeVIS_test)),axis=2)
    decision_function_both = np.mean(decision_function_both,axis=2)
    decisions = np.where(decision_function_both>0,0,1) #0:positive, 1:negative
    for s in range(test_parcel_num):
        predictions = []
        # loop through all the classifiers:
        for vc in range(len(votes)):
            index = decisions[s,vc]
            #index = decisions_DFC[s,vc]
            #predictions.append(votes[vc,index])
            #index = decisions_VIS[s,vc]
            predictions.append(votes[vc,index])
        
        # find the category with most votes
        null_pred_Both[s,c] = int(st.mode(predictions))
        
    null_recall_Both[c,:] = recall_score(label_test, null_pred_Both[:,c], average=None)
    null_score_Both.append(f1_score(label_test, null_pred_Both[:,c], average='macro'))
    print(str(c)+' f-1 score for BOTH is: '+str(null_score_Both[c]))

0 f-1 score for Lake DFC is: 0.19876758947966375
0 f-1 score for Lake VIS is: 0.09000523286237572
0 f-1 score for BOTH is: 0.08100740296118447
1 f-1 score for Lake DFC is: 0.12035714285714286
1 f-1 score for Lake VIS is: 0.07277167277167276
1 f-1 score for BOTH is: 0.08442632636181023
2 f-1 score for Lake DFC is: 0.1197891314083218
2 f-1 score for Lake VIS is: 0.17918716989924421
2 f-1 score for BOTH is: 0.12811791383219953
3 f-1 score for Lake DFC is: 0.15362811791383219
3 f-1 score for Lake VIS is: 0.17261887616578692
3 f-1 score for BOTH is: 0.1666034362462934
4 f-1 score for Lake DFC is: 0.1569509855224141
4 f-1 score for Lake VIS is: 0.12142857142857144
4 f-1 score for BOTH is: 0.10163170163170163
5 f-1 score for Lake DFC is: 0.2581227706393562
5 f-1 score for Lake VIS is: 0.23281782629608716
5 f-1 score for BOTH is: 0.1903050456787741
6 f-1 score for Lake DFC is: 0.15837482328354538
6 f-1 score for Lake VIS is: 0.1608328179756751
6 f-1 score for BOTH is: 0.19131652661064427
7 f-1

58 f-1 score for Lake VIS is: 0.1708314904909332
58 f-1 score for BOTH is: 0.15274518898468478
59 f-1 score for Lake DFC is: 0.1395589704413234
59 f-1 score for Lake VIS is: 0.06211484593837535
59 f-1 score for BOTH is: 0.144011544011544
60 f-1 score for Lake DFC is: 0.0897358943577431
60 f-1 score for Lake VIS is: 0.04986917844060701
60 f-1 score for BOTH is: 0.09047619047619047
61 f-1 score for Lake DFC is: 0.21391849529780563
61 f-1 score for Lake VIS is: 0.06010325078648059
61 f-1 score for BOTH is: 0.14338728903946296
62 f-1 score for Lake DFC is: 0.12040816326530612
62 f-1 score for Lake VIS is: 0.07030873888016745
62 f-1 score for BOTH is: 0.15705009276437848
63 f-1 score for Lake DFC is: 0.11995328343154431
63 f-1 score for Lake VIS is: 0.1256235827664399
63 f-1 score for BOTH is: 0.09913043478260872
64 f-1 score for Lake DFC is: 0.10381364667078954
64 f-1 score for Lake VIS is: 0.045068027210884355
64 f-1 score for BOTH is: 0.10565953654188949
65 f-1 score for Lake DFC is: 0.1

114 f-1 score for BOTH is: 0.0596905428838202
115 f-1 score for Lake DFC is: 0.130718487861345
115 f-1 score for Lake VIS is: 0.10509490509490509
115 f-1 score for BOTH is: 0.11782696148021225
116 f-1 score for Lake DFC is: 0.07888407888407889
116 f-1 score for Lake VIS is: 0.2231050522975989
116 f-1 score for BOTH is: 0.1323002279524019
117 f-1 score for Lake DFC is: 0.13721446473326174
117 f-1 score for Lake VIS is: 0.12332852332852332
117 f-1 score for BOTH is: 0.12851663742420047
118 f-1 score for Lake DFC is: 0.12252901160464187
118 f-1 score for Lake VIS is: 0.060317460317460325
118 f-1 score for BOTH is: 0.11598639455782313
119 f-1 score for Lake DFC is: 0.059319727891156464
119 f-1 score for Lake VIS is: 0.027891156462585033
119 f-1 score for BOTH is: 0.08390022675736963
120 f-1 score for Lake DFC is: 0.17976190476190476
120 f-1 score for Lake VIS is: 0.18310345669368225
120 f-1 score for BOTH is: 0.09287010630023333
121 f-1 score for Lake DFC is: 0.1404707044107966
121 f-1 sco

174 f-1 score for Lake DFC is: 0.058062434459949984
174 f-1 score for Lake VIS is: 0.1084167000133387
174 f-1 score for BOTH is: 0.12011466794075489
175 f-1 score for Lake DFC is: 0.20367914568204193
175 f-1 score for Lake VIS is: 0.14754760203676612
175 f-1 score for BOTH is: 0.16187221300003254
176 f-1 score for Lake DFC is: 0.07878707038370905
176 f-1 score for Lake VIS is: 0.07571347724149083
176 f-1 score for BOTH is: 0.12202982202982203
177 f-1 score for Lake DFC is: 0.16001055594651656
177 f-1 score for Lake VIS is: 0.09871794871794871
177 f-1 score for BOTH is: 0.15185863819211895
178 f-1 score for Lake DFC is: 0.18995865012671737
178 f-1 score for Lake VIS is: 0.1473922902494331
178 f-1 score for BOTH is: 0.22590686624300066
179 f-1 score for Lake DFC is: 0.07092766980565017
179 f-1 score for Lake VIS is: 0.1578184591914569
179 f-1 score for BOTH is: 0.15164835164835164
180 f-1 score for Lake DFC is: 0.17362500555777866
180 f-1 score for Lake VIS is: 0.14669205063941906
180 f-

235 f-1 score for Lake DFC is: 0.06340718105423988
235 f-1 score for Lake VIS is: 0.12722897403348532
235 f-1 score for BOTH is: 0.09320091673032849
236 f-1 score for Lake DFC is: 0.19925048496477069
236 f-1 score for Lake VIS is: 0.09447428094044634
236 f-1 score for BOTH is: 0.17746845746845746
237 f-1 score for Lake DFC is: 0.16984126984126985
237 f-1 score for Lake VIS is: 0.1762215997510115
237 f-1 score for BOTH is: 0.15262776439247028
238 f-1 score for Lake DFC is: 0.16301484230055657
238 f-1 score for Lake VIS is: 0.18886258787736618
238 f-1 score for BOTH is: 0.14189017504234896
239 f-1 score for Lake DFC is: 0.10397768615086103
239 f-1 score for Lake VIS is: 0.1657884026305079
239 f-1 score for BOTH is: 0.2059059987631416
240 f-1 score for Lake DFC is: 0.13770470131372387
240 f-1 score for Lake VIS is: 0.14452380952380953
240 f-1 score for BOTH is: 0.13736263736263735
241 f-1 score for Lake DFC is: 0.09652873006910274
241 f-1 score for Lake VIS is: 0.11458095997569681
241 f-1

...

707 f-1 score for BOTH is: 0.07916499933306656
708 f-1 score for Lake DFC is: 0.1636816599549519
708 f-1 score for Lake VIS is: 0.14473989595838335
708 f-1 score for BOTH is: 0.16763038548752834
709 f-1 score for Lake DFC is: 0.19950469950469948
709 f-1 score for Lake VIS is: 0.1652474509617367
709 f-1 score for BOTH is: 0.16984126984126985
710 f-1 score for Lake DFC is: 0.20496696286169971
710 f-1 score for Lake VIS is: 0.15127844241144733
710 f-1 score for BOTH is: 0.13388278388278388
711 f-1 score for Lake DFC is: 0.13512569302042984
711 f-1 score for Lake VIS is: 0.14434251961654224
711 f-1 score for BOTH is: 0.12726561889123467
712 f-1 score for Lake DFC is: 0.1989486703772418
712 f-1 score for Lake VIS is: 0.15258467023172909
712 f-1 score for BOTH is: 0.18748657357679915
713 f-1 score for Lake DFC is: 0.1111859569002426
713 f-1 score for Lake VIS is: 0.0977891156462585
713 f-1 score for BOTH is: 0.14361101861101863
714 f-1 score for Lake DFC is: 0.08587228941920018
714 f-1 score

764 f-1 score for BOTH is: 0.12671957671957673
765 f-1 score for Lake DFC is: 0.21906456671262425
765 f-1 score for Lake VIS is: 0.09976190476190476
765 f-1 score for BOTH is: 0.10760421390081094
766 f-1 score for Lake DFC is: 0.1640981240981241
766 f-1 score for Lake VIS is: 0.07857142857142858
766 f-1 score for BOTH is: 0.09186369186369188
767 f-1 score for Lake DFC is: 0.0845330439868255
767 f-1 score for Lake VIS is: 0.1677675772503359
767 f-1 score for BOTH is: 0.14155697243932536
768 f-1 score for Lake DFC is: 0.17499469129903913
768 f-1 score for Lake VIS is: 0.22208469892680419
768 f-1 score for BOTH is: 0.2088702147525677
769 f-1 score for Lake DFC is: 0.07440038222646918
769 f-1 score for Lake VIS is: 0.16631737827390003
769 f-1 score for BOTH is: 0.1004995004995005
770 f-1 score for Lake DFC is: 0.13834671799754386
770 f-1 score for Lake VIS is: 0.10897576421873098
770 f-1 score for BOTH is: 0.13752774794128178
771 f-1 score for Lake DFC is: 0.1386681945505475
771 f-1 score 

825 f-1 score for Lake DFC is: 0.16785714285714287
825 f-1 score for Lake VIS is: 0.09916457811194654
825 f-1 score for BOTH is: 0.1761583493967302
826 f-1 score for Lake DFC is: 0.08862006340997938
826 f-1 score for Lake VIS is: 0.12728249194414606
826 f-1 score for BOTH is: 0.0944996180290298
827 f-1 score for Lake DFC is: 0.09371021135727019
827 f-1 score for Lake VIS is: 0.06787878787878789
827 f-1 score for BOTH is: 0.08699843573793155
828 f-1 score for Lake DFC is: 0.1457875457875458
828 f-1 score for Lake VIS is: 0.0825370281892021
828 f-1 score for BOTH is: 0.07123233908948194
829 f-1 score for Lake DFC is: 0.08917748917748917
829 f-1 score for Lake VIS is: 0.1513225870058168
829 f-1 score for BOTH is: 0.12990777338603426
830 f-1 score for Lake DFC is: 0.07353081700907789
830 f-1 score for Lake VIS is: 0.18241341991341992
830 f-1 score for BOTH is: 0.1051168293404318
831 f-1 score for Lake DFC is: 0.1891304347826087
831 f-1 score for Lake VIS is: 0.13895255071725662
831 f-1 sco

885 f-1 score for Lake DFC is: 0.11954648526077097
885 f-1 score for Lake VIS is: 0.16598639455782313
885 f-1 score for BOTH is: 0.16747062461348175
886 f-1 score for Lake DFC is: 0.07407273254129236
886 f-1 score for Lake VIS is: 0.09555206698063842
886 f-1 score for BOTH is: 0.08109243697478993
887 f-1 score for Lake DFC is: 0.10627281215516511
887 f-1 score for Lake VIS is: 0.07379430032882718
887 f-1 score for BOTH is: 0.11738403980966912
888 f-1 score for Lake DFC is: 0.1524446142093201
888 f-1 score for Lake VIS is: 0.05466964113580656
888 f-1 score for BOTH is: 0.09400452077797615
889 f-1 score for Lake DFC is: 0.057371200228343085
889 f-1 score for Lake VIS is: 0.15006845006845007
889 f-1 score for BOTH is: 0.16769230769230772
890 f-1 score for Lake DFC is: 0.10963718820861679
890 f-1 score for Lake VIS is: 0.07959183673469387
890 f-1 score for BOTH is: 0.11515151515151513
891 f-1 score for Lake DFC is: 0.0725491147222896
891 f-1 score for Lake VIS is: 0.09519632414369257
891 f

944 f-1 score for BOTH is: 0.1326764038948913
945 f-1 score for Lake DFC is: 0.08336316982933524
945 f-1 score for Lake VIS is: 0.02380952380952381
945 f-1 score for BOTH is: 0.11640565317035907
946 f-1 score for Lake DFC is: 0.14936974789915966
946 f-1 score for Lake VIS is: 0.12375366568914958
946 f-1 score for BOTH is: 0.07039473684210527
947 f-1 score for Lake DFC is: 0.125250100040016
947 f-1 score for Lake VIS is: 0.10528138528138528
947 f-1 score for BOTH is: 0.14377433010973384
948 f-1 score for Lake DFC is: 0.10543884220354809
948 f-1 score for Lake VIS is: 0.1511312622214878
948 f-1 score for BOTH is: 0.18518518518518517
949 f-1 score for Lake DFC is: 0.17030452385143463
949 f-1 score for Lake VIS is: 0.11478808914870295
949 f-1 score for BOTH is: 0.16258485759764535
950 f-1 score for Lake DFC is: 0.13450774612693653
950 f-1 score for Lake VIS is: 0.08027950310559007
950 f-1 score for BOTH is: 0.12077148239067281
951 f-1 score for Lake DFC is: 0.11993720565149138
951 f-1 scor

In [44]:
# save the results
with open(net_enrich_dir+'/1000NullsBalancedPermEnsembeled_Left_2.pkl', 'wb') as f:
    pickle.dump([null_test, sample_test,null_pred_Both,null_score_Both,null_recall_Both,
                 null_pred_DFC, null_score_DFC, null_recall_DFC,
                 null_pred_VIS, null_score_VIS, null_recall_VIS], f)

#### 1000 spin-test nulls
Null datas are generated by spinning the network labels while controlling for the spatial auto-correlation.

In [39]:
clf_num = 1000
parcel_num = 191
test_parcel_num = 58
net_num = 7
schaefer400_7Networks_mat_left = schaefer400_7Networks_mat[:191,:]
LakeDFC_schaefer400_mat_left = LakeDFC_schaefer400_mat[:,:191]
LakeVIS_schaefer400_mat_left = LakeVIS_schaefer400_mat[:,:191]

null_test = np.zeros((test_parcel_num,clf_num))
sample_test = np.zeros((test_parcel_num,clf_num))

null_pred_Both = np.zeros((test_parcel_num,clf_num))
null_score_Both = []
null_recall_Both = np.zeros((clf_num,net_num))

null_pred_DFC = np.zeros((test_parcel_num,clf_num))
null_score_DFC = []
null_recall_DFC = np.zeros((clf_num,net_num))

null_pred_VIS = np.zeros((test_parcel_num,clf_num))
null_score_VIS = []
null_recall_VIS = np.zeros((clf_num,net_num))
for c in range(clf_num):
    
    # Step0: Get the spin-tested label
    schaefer400_7Networks_perm = Null_7NetLabel_mat[:191,c]

    # Step1: Split 70% for train and 30% for test
    train, test = train_test_split(list(range(parcel_num)), test_size=0.3, stratify=schaefer400_7Networks_perm,random_state=c+1000)
    label_train = schaefer400_7Networks_perm[train]
    label_test = schaefer400_7Networks_perm[test]
    # save the permutated lable
    #null_test[:,c] = label_test[:,0]
    null_test[:,c] = label_test
    # save the ID of the tested parcels
    sample_test[:,c] = test

    LakeDFC_train = np.transpose(LakeDFC_schaefer400_mat_left[:,train])
    LakeDFC_test = np.transpose(LakeDFC_schaefer400_mat_left[:,test])

    LakeVIS_train = np.transpose(LakeVIS_schaefer400_mat_left[:,train])
    LakeVIS_test = np.transpose(LakeVIS_schaefer400_mat_left[:,test])


    ### Lake DFC ###
    # Step2: Random draw a hyper-combo then train the classifer
    # find the best hyper-combo
    np.random.seed(c+1000)
    best_idx = np.random.randint(1000, size=1)[0]

    # Step3: Train the classifier
    classifier = SVC(class_weight='balanced',
                     random_state=c+1000,
                     C=C_best_DFC[best_idx],
                     gamma=gamma_best_DFC[best_idx],
                     degree=degree_best_DFC[best_idx],
                     kernel=kernel_best_DFC[best_idx],
                     decision_function_shape='ovo')
    final_clf_DFC = classifier.fit(LakeDFC_train, label_train)

    # Test the classifier
    null_pred_DFC[:,c] = final_clf_DFC.predict(LakeDFC_test)
    null_score_DFC.append(f1_score(label_test, null_pred_DFC[:,c], average='macro'))
    print(str(c)+' f-1 score for Lake DFC is: '+str(null_score_DFC[c]))
    null_recall_DFC[c,:] = recall_score(label_test, null_pred_DFC[:,c], average=None)


    ### Lake VIS ###
    # Step2: Random draw a hyper-combo then train the classifer
    # find the best hyper-combo
    np.random.seed(c+1000)
    best_idx = np.random.randint(1000, size=1)[0]


    # Step3: Train the classifier
    classifier = SVC(class_weight='balanced',
                     random_state=c+1000,
                     C=C_best_VIS[best_idx],
                     gamma=gamma_best_VIS[best_idx],
                     degree=degree_best_VIS[best_idx],
                     kernel=kernel_best_VIS[best_idx],
                     decision_function_shape='ovo')
    final_clf_VIS = classifier.fit(LakeVIS_train, label_train)

    # Test the classifier
    null_pred_VIS[:,c] = final_clf_VIS.predict(LakeVIS_test)
    null_score_VIS.append(f1_score(label_test, null_pred_VIS[:,c], average='macro'))
    print(str(c)+' f-1 score for Lake VIS is: '+str(null_score_VIS[c]))
    null_recall_VIS[c,:] = recall_score(label_test, null_pred_VIS[:,c], average=None)
    
    ### Ensembling VIS and DFC ###
    decision_function_both = np.stack((final_clf_DFC.decision_function(LakeDFC_test),final_clf_VIS.decision_function(LakeVIS_test)),axis=2)
    decision_function_both = np.mean(decision_function_both,axis=2)
    decisions = np.where(decision_function_both>0,0,1) #0:positive, 1:negative
    for s in range(test_parcel_num):
        predictions = []
        # loop through all the classifiers:
        for vc in range(len(votes)):
            index = decisions[s,vc]
            #index = decisions_DFC[s,vc]
            #predictions.append(votes[vc,index])
            #index = decisions_VIS[s,vc]
            predictions.append(votes[vc,index])
        
        # find the category with most votes
        null_pred_Both[s,c] = int(st.mode(predictions))
        
    null_recall_Both[c,:] = recall_score(label_test, null_pred_Both[:,c], average=None)
    null_score_Both.append(f1_score(label_test, null_pred_Both[:,c], average='macro'))
    print(str(c)+' f-1 score for BOTH is: '+str(null_score_Both[c]))

0 f-1 score for Lake DFC is: 0.25969157172850993
0 f-1 score for Lake VIS is: 0.2488361174429286
0 f-1 score for BOTH is: 0.3276158623984711
1 f-1 score for Lake DFC is: 0.3224140938426653
1 f-1 score for Lake VIS is: 0.38579071304635215
1 f-1 score for BOTH is: 0.3263373263373263
2 f-1 score for Lake DFC is: 0.19445939666189058
2 f-1 score for Lake VIS is: 0.3014009799724086
2 f-1 score for BOTH is: 0.26818733490397656
3 f-1 score for Lake DFC is: 0.28568173680955633
3 f-1 score for Lake VIS is: 0.23386081705409434
3 f-1 score for BOTH is: 0.16737773206597617
4 f-1 score for Lake DFC is: 0.3543290043290043
4 f-1 score for Lake VIS is: 0.2536281179138322
4 f-1 score for BOTH is: 0.2551820728291316
5 f-1 score for Lake DFC is: 0.18658534448008135
5 f-1 score for Lake VIS is: 0.15407910802647645
5 f-1 score for BOTH is: 0.1807081807081807
6 f-1 score for Lake DFC is: 0.1364172335600907
6 f-1 score for Lake VIS is: 0.21705605319050697
6 f-1 score for BOTH is: 0.2065062706741867
7 f-1 scor

61 f-1 score for Lake DFC is: 0.37313344918386937
61 f-1 score for Lake VIS is: 0.39125530595013125
61 f-1 score for BOTH is: 0.3378003983594046
62 f-1 score for Lake DFC is: 0.18360934117236635
62 f-1 score for Lake VIS is: 0.20529375386518245
62 f-1 score for BOTH is: 0.23469994058229354
63 f-1 score for Lake DFC is: 0.29107551487414185
63 f-1 score for Lake VIS is: 0.27700136464842345
63 f-1 score for BOTH is: 0.26239950525664807
64 f-1 score for Lake DFC is: 0.1976030099888339
64 f-1 score for Lake VIS is: 0.2091664948807806
64 f-1 score for BOTH is: 0.2636720005141058
65 f-1 score for Lake DFC is: 0.1494124922696351
65 f-1 score for Lake VIS is: 0.1536590393733251
65 f-1 score for BOTH is: 0.14677386105957538
66 f-1 score for Lake DFC is: 0.16817171476614204
66 f-1 score for Lake VIS is: 0.19801993261072545
66 f-1 score for BOTH is: 0.17957439386010812
67 f-1 score for Lake DFC is: 0.2550911821098157
67 f-1 score for Lake VIS is: 0.15557720057720056
67 f-1 score for BOTH is: 0.290

127 f-1 score for Lake DFC is: 0.34034854034854034
127 f-1 score for Lake VIS is: 0.22873240607230752
127 f-1 score for BOTH is: 0.173633969716919
128 f-1 score for Lake DFC is: 0.27680272108843534
128 f-1 score for Lake VIS is: 0.42961334731521067
128 f-1 score for BOTH is: 0.3674169189206783
129 f-1 score for Lake DFC is: 0.16785714285714287
129 f-1 score for Lake VIS is: 0.19347523955818885
129 f-1 score for BOTH is: 0.19482184482184484
130 f-1 score for Lake DFC is: 0.2633720154728558
130 f-1 score for Lake VIS is: 0.26571536367801224
130 f-1 score for BOTH is: 0.31283676640819497
131 f-1 score for Lake DFC is: 0.2429379195683543
131 f-1 score for Lake VIS is: 0.20545591974163407
131 f-1 score for BOTH is: 0.23694173321502515
132 f-1 score for Lake DFC is: 0.18121660258306221
132 f-1 score for Lake VIS is: 0.31736239950525663
132 f-1 score for BOTH is: 0.26889935461364034
133 f-1 score for Lake DFC is: 0.2422335600907029
133 f-1 score for Lake VIS is: 0.22693833943833946
133 f-1 sc

185 f-1 score for Lake VIS is: 0.2394927536231884
185 f-1 score for BOTH is: 0.2729455418531049
186 f-1 score for Lake DFC is: 0.2157978981066111
186 f-1 score for Lake VIS is: 0.2294587400177462
186 f-1 score for BOTH is: 0.18849262192915445
187 f-1 score for Lake DFC is: 0.2931065759637188
187 f-1 score for Lake VIS is: 0.1970058926580666
187 f-1 score for BOTH is: 0.36750973893831035
188 f-1 score for Lake DFC is: 0.32293818713256056
188 f-1 score for Lake VIS is: 0.2454838394688019
188 f-1 score for BOTH is: 0.2974100409394527
189 f-1 score for Lake DFC is: 0.18610171341263776
189 f-1 score for Lake VIS is: 0.16547947681488054
189 f-1 score for BOTH is: 0.30030072635114646
190 f-1 score for Lake DFC is: 0.10103445426753697
190 f-1 score for Lake VIS is: 0.0810989010989011
190 f-1 score for BOTH is: 0.09634700275094364
191 f-1 score for Lake DFC is: 0.3147225323695912
191 f-1 score for Lake VIS is: 0.20324847110561395
191 f-1 score for BOTH is: 0.24712612317654334
192 f-1 score for 

242 f-1 score for Lake DFC is: 0.2948563980822046
242 f-1 score for Lake VIS is: 0.27987617972090023
242 f-1 score for BOTH is: 0.32520365472310553
243 f-1 score for Lake DFC is: 0.304421768707483
243 f-1 score for Lake VIS is: 0.27513962106581763
243 f-1 score for BOTH is: 0.2536507936507936
244 f-1 score for Lake DFC is: 0.24904064189778471
244 f-1 score for Lake VIS is: 0.2661449407444831
244 f-1 score for BOTH is: 0.2267000835421888
245 f-1 score for Lake DFC is: 0.16641156462585036
245 f-1 score for Lake VIS is: 0.30334577609154606
245 f-1 score for BOTH is: 0.2192513368983957
246 f-1 score for Lake DFC is: 0.23940886699507385
246 f-1 score for Lake VIS is: 0.2592764378478664
246 f-1 score for BOTH is: 0.32567693744164333
247 f-1 score for Lake DFC is: 0.1843744690344111
247 f-1 score for Lake VIS is: 0.3228971638582622
247 f-1 score for BOTH is: 0.31998763141620296
248 f-1 score for Lake DFC is: 0.09266945644654623
248 f-1 score for Lake VIS is: 0.20442089879430034
248 f-1 score 

...

727 f-1 score for Lake VIS is: 0.37925518385491847
727 f-1 score for BOTH is: 0.375327812284334
728 f-1 score for Lake DFC is: 0.17153818048958716
728 f-1 score for Lake VIS is: 0.2701429854238346
728 f-1 score for BOTH is: 0.2521871779014636
729 f-1 score for Lake DFC is: 0.25285999567291034
729 f-1 score for Lake VIS is: 0.1860275689223058
729 f-1 score for BOTH is: 0.23273037666630342
730 f-1 score for Lake DFC is: 0.23072399613753
730 f-1 score for Lake VIS is: 0.2197682491800139
730 f-1 score for BOTH is: 0.22371615312791787
731 f-1 score for Lake DFC is: 0.2880742649163701
731 f-1 score for Lake VIS is: 0.3085644514215943
731 f-1 score for BOTH is: 0.32650946936661224
732 f-1 score for Lake DFC is: 0.19224440348221666
732 f-1 score for Lake VIS is: 0.2783210985543642
732 f-1 score for BOTH is: 0.19483491337038247
733 f-1 score for Lake DFC is: 0.20812520812520813
733 f-1 score for Lake VIS is: 0.17379259396066118
733 f-1 score for BOTH is: 0.16607704089593398
734 f-1 score for La

787 f-1 score for Lake VIS is: 0.2693800008041494
787 f-1 score for BOTH is: 0.25409471480900053
788 f-1 score for Lake DFC is: 0.1834733893557423
788 f-1 score for Lake VIS is: 0.2071952891828668
788 f-1 score for BOTH is: 0.3076975405546834
789 f-1 score for Lake DFC is: 0.26529944575057357
789 f-1 score for Lake VIS is: 0.2510942657922739
789 f-1 score for BOTH is: 0.27625413801884385
790 f-1 score for Lake DFC is: 0.15191511387163562
790 f-1 score for Lake VIS is: 0.22029661160095942
790 f-1 score for BOTH is: 0.19551820728291316
791 f-1 score for Lake DFC is: 0.26732273937242884
791 f-1 score for Lake VIS is: 0.27224489795918366
791 f-1 score for BOTH is: 0.2010351966873706
792 f-1 score for Lake DFC is: 0.17211802262134238
792 f-1 score for Lake VIS is: 0.08905538905538905
792 f-1 score for BOTH is: 0.06932854113305241
793 f-1 score for Lake DFC is: 0.29049093321539143
793 f-1 score for Lake VIS is: 0.24109933828603908
793 f-1 score for BOTH is: 0.28303442589156874
794 f-1 score 

847 f-1 score for Lake DFC is: 0.2791097827384195
847 f-1 score for Lake VIS is: 0.29417130488559057
847 f-1 score for BOTH is: 0.22956567242281528
848 f-1 score for Lake DFC is: 0.36655642494151813
848 f-1 score for Lake VIS is: 0.3668027210884354
848 f-1 score for BOTH is: 0.3813088871912402
849 f-1 score for Lake DFC is: 0.23292045428691394
849 f-1 score for Lake VIS is: 0.20271658166395015
849 f-1 score for BOTH is: 0.17210170781599352
850 f-1 score for Lake DFC is: 0.3035321500089116
850 f-1 score for Lake VIS is: 0.3235722248765727
850 f-1 score for BOTH is: 0.3851783137497423
851 f-1 score for Lake DFC is: 0.3525327674395998
851 f-1 score for Lake VIS is: 0.3431320002748574
851 f-1 score for BOTH is: 0.2749615975422427
852 f-1 score for Lake DFC is: 0.202915948988291
852 f-1 score for Lake VIS is: 0.17627464382882338
852 f-1 score for BOTH is: 0.13481948334889512
853 f-1 score for Lake DFC is: 0.3942052718523307
853 f-1 score for Lake VIS is: 0.3127061935885465
853 f-1 score for

903 f-1 score for Lake DFC is: 0.3268644226185137
903 f-1 score for Lake VIS is: 0.25253579692746664
903 f-1 score for BOTH is: 0.36209483793517405
904 f-1 score for Lake DFC is: 0.24049015500628404
904 f-1 score for Lake VIS is: 0.2000523286237572
904 f-1 score for BOTH is: 0.21268958728886597
905 f-1 score for Lake DFC is: 0.26968269825412683
905 f-1 score for Lake VIS is: 0.2399626517273576
905 f-1 score for BOTH is: 0.22850082248578488
906 f-1 score for Lake DFC is: 0.20586080586080585
906 f-1 score for Lake VIS is: 0.16146310211527606
906 f-1 score for BOTH is: 0.23645093782393553
907 f-1 score for Lake DFC is: 0.3214162028447742
907 f-1 score for Lake VIS is: 0.29064625850340137
907 f-1 score for BOTH is: 0.3102240896358543
908 f-1 score for Lake DFC is: 0.15675435675435676
908 f-1 score for Lake VIS is: 0.27366163856847087
908 f-1 score for BOTH is: 0.26101808244665387
909 f-1 score for Lake DFC is: 0.2530194873648384
909 f-1 score for Lake VIS is: 0.22218733647305075
909 f-1 sc

960 f-1 score for Lake DFC is: 0.1972651408741634
960 f-1 score for Lake VIS is: 0.17511004401760705
960 f-1 score for BOTH is: 0.20688121402407117
961 f-1 score for Lake DFC is: 0.15629375162043038
961 f-1 score for Lake VIS is: 0.1674018709487817
961 f-1 score for BOTH is: 0.1496296296296296
962 f-1 score for Lake DFC is: 0.20389266817838245
962 f-1 score for Lake VIS is: 0.2404490298534873
962 f-1 score for BOTH is: 0.25009593581022155
963 f-1 score for Lake DFC is: 0.20363115145723842
963 f-1 score for Lake VIS is: 0.13272635683501557
963 f-1 score for BOTH is: 0.22566368330986022
964 f-1 score for Lake DFC is: 0.23575255154202526
964 f-1 score for Lake VIS is: 0.2133254109724698
964 f-1 score for BOTH is: 0.21953324453324455
965 f-1 score for Lake DFC is: 0.2409229940670103
965 f-1 score for Lake VIS is: 0.19727891156462585
965 f-1 score for BOTH is: 0.27936507936507937
966 f-1 score for Lake DFC is: 0.25579154738818605
966 f-1 score for Lake VIS is: 0.2640610241291467
966 f-1 sco

In [40]:
# save the results
with open(net_enrich_dir+'/1000NullsBalancedSpinEnsembeled_Left_2.pkl', 'wb') as f:
    pickle.dump([null_test, sample_test,null_pred_Both,null_score_Both,null_recall_Both,
                 null_pred_DFC, null_score_DFC, null_recall_DFC,
                 null_pred_VIS, null_score_VIS, null_recall_VIS], f)

In [41]:
# load the results
null_test, sample_test, null_pred_Both,null_score_Both,null_recall_Both,null_pred_DFC, null_score_DFC, null_recall_DFC, null_pred_VIS, null_score_VIS, null_recall_VIS = pickle.load(open(net_enrich_dir+'/1000NullsBalancedSpinEnsembeled_Left_2.pkl',"rb"))

### Both Hemispheres
#### 1000 classifier

In [48]:
clf_num = 1000
parcel_num = 339
test_parcel_num = 102
net_num = 7

clf_test = np.zeros((test_parcel_num,clf_num))
sample_test = np.zeros((test_parcel_num,clf_num))
clf_pred_Both = np.zeros((test_parcel_num,clf_num))
clf_score_Both = []
clf_recall_Both = np.zeros((clf_num,net_num))

clf_pred_DFC = np.zeros((test_parcel_num,clf_num))
clf_score_DFC = []
clf_recall_DFC = np.zeros((clf_num,net_num))
C_best_DFC = []
gamma_best_DFC = []
kernel_best_DFC = []
degree_best_DFC = []

clf_pred_VIS = np.zeros((test_parcel_num,clf_num))
clf_score_VIS = []
clf_recall_VIS = np.zeros((clf_num,net_num))
C_best_VIS = []
gamma_best_VIS = []
kernel_best_VIS = []
degree_best_VIS = []
for c in range(clf_num):
    
    # Step1: Split 70% for train and 30% for test
    train, test = train_test_split(list(range(parcel_num)), test_size=0.3, stratify=schaefer400_7Networks_mat,random_state=c)
    label_train = schaefer400_7Networks_mat[train]
    label_test = schaefer400_7Networks_mat[test]
    # save the real category of the tested parcels
    clf_test[:,c] = label_test[:,0]
    # save the ID of the tested parcels
    sample_test[:,c] = test

    LakeDFC_train = np.transpose(LakeDFC_schaefer400_mat[:,train])
    LakeDFC_test = np.transpose(LakeDFC_schaefer400_mat[:,test])

    LakeVIS_train = np.transpose(LakeVIS_schaefer400_mat[:,train])
    LakeVIS_test = np.transpose(LakeVIS_schaefer400_mat[:,test])
    
    
    ### Lake DFC ###
    # Step2: Nested 3-fold cross-validation for hyperparameter tunning
    # Outter loop: test the classifier performance
    kf_outer = StratifiedKFold(n_splits=3,shuffle=True,random_state=c)
    clf_score = []
    C_best = []
    gamma_best = []
    kernel_best = []
    degree_best = []
    for train_outer, test_outer in kf_outer.split(LakeDFC_train, label_train):
        # Pull out the sample data
        label_train_outer = label_train[train_outer]
        label_test_outer = label_train[test_outer]
    
        DFC_train_outer = LakeDFC_train[train_outer,:]
        DFC_test_outer = LakeDFC_train[test_outer,:]

        ###### Inner loop: tune the hyper-parameter ######
        kf_inner = StratifiedKFold(n_splits=2,shuffle=True,random_state=c)
    
        # Search over different regularization parameters: smaller values specify stronger regularization.
        parameters = {'C':[1e-05, 1e-03, 0.1, 10, 1e03, 1e05],
                      'gamma':[1e-03, 0.1, 10, 1e03],
                      'degree':[2,3,4],
                      'kernel':['poly', 'rbf']}
        
        inner_clf = GridSearchCV(
            SVC(class_weight='balanced',random_state=c),
            parameters,
            cv=kf_inner,
            scoring='f1_macro',
            return_train_score=True)
        inner_clf.fit(DFC_train_outer, label_train_outer)
        ###### Inner loop: tune the hyper-parameter ######
    
        # Find the best hyperparameter
        C_best.append(inner_clf.best_params_['C'])
        gamma_best.append(inner_clf.best_params_['gamma'])
        kernel_best.append(inner_clf.best_params_['kernel'])
        degree_best.append(inner_clf.best_params_['degree'])
    
        # Train the classifier with the best hyperparameter using training and validation set
        classifier = SVC(class_weight='balanced',
                         random_state=c,
                         C=inner_clf.best_params_['C'],
                         gamma=inner_clf.best_params_['gamma'],
                         degree=inner_clf.best_params_['degree'],
                         kernel=inner_clf.best_params_['kernel'])
        outer_clf = classifier.fit(DFC_train_outer, label_train_outer)
        
        # Validate the classifier
        score = outer_clf.score(DFC_test_outer, label_test_outer)
        clf_score.append(score)
    
    # find the best hyper-combo
    best_idx = clf_score.index(max(clf_score))
    
    C_best_DFC.append(C_best[best_idx])
    gamma_best_DFC.append(gamma_best[best_idx])
    kernel_best_DFC.append(kernel_best[best_idx])
    degree_best_DFC.append(degree_best[best_idx])
    print ('Best cost value:', C_best_DFC[c])
    print ('Best gamma:', gamma_best_DFC[c])
    print ('Best kernel:', kernel_best_DFC[c])
    print ('Best degree:', degree_best_DFC[c])
    
    classifier = SVC(class_weight='balanced',
                     random_state=c,
                     C=C_best[best_idx],
                     gamma=gamma_best[best_idx],
                     kernel=kernel_best[best_idx],
                     degree=degree_best[best_idx],
                     decision_function_shape='ovo')
    final_clf_DFC = classifier.fit(LakeDFC_train, label_train)
    
    # Test the classifier
    clf_pred_DFC[:,c] = final_clf_DFC.predict(LakeDFC_test)
    clf_score_DFC.append(f1_score(label_test, clf_pred_DFC[:,c], average='macro'))
    print(str(c)+' f-1 score for Lake DFC is: '+str(clf_score_DFC[c]))
    clf_recall_DFC[c,:] = recall_score(label_test, clf_pred_DFC[:,c], average=None)

    
    ### Lake VIS ###
    # Step2: Nested 3-fold cross-validation for hyperparameter tunning
    # Outter loop: test the classifier performance
    kf_outer = StratifiedKFold(n_splits=3,shuffle=True,random_state=c)
    clf_score = []
    C_best = []
    gamma_best = []
    kernel_best = []
    degree_best = []
    for train_outer, test_outer in kf_outer.split(LakeVIS_train, label_train):
        # Pull out the sample data
        label_train_outer = label_train[train_outer]
        label_test_outer = label_train[test_outer]
    
        VIS_train_outer = LakeVIS_train[train_outer,:]
        VIS_test_outer = LakeVIS_train[test_outer,:]

        ###### Inner loop: tune the hyper-parameter ######
        kf_inner = StratifiedKFold(n_splits=3,shuffle=True,random_state=c)
    
        # Search over different regularization parameters: smaller values specify stronger regularization.
        parameters = {'C':[1e-05, 1e-03, 0.1, 10, 1e03, 1e05],
                      'gamma':[1e-03, 0.1, 10, 1e03],
                      'degree':[2,3,4],
                      'kernel':['poly', 'rbf']}
    
        inner_clf = GridSearchCV(
            SVC(class_weight='balanced',random_state=c),
            parameters,
            cv=kf_inner,
            scoring='f1_macro',
            return_train_score=True)
        inner_clf.fit(VIS_train_outer, label_train_outer)
        ###### Inner loop: tune the hyper-parameter ######
    
        # Find the best hyperparameter
        C_best.append(inner_clf.best_params_['C'])
        gamma_best.append(inner_clf.best_params_['gamma'])
        kernel_best.append(inner_clf.best_params_['kernel'])
        degree_best.append(inner_clf.best_params_['degree'])
    
        # Train the classifier with the best hyperparameter using training and validation set
        classifier = SVC(class_weight='balanced',
                         random_state=c,
                         C=inner_clf.best_params_['C'],
                         gamma=inner_clf.best_params_['gamma'],
                         degree=inner_clf.best_params_['degree'],
                         kernel=inner_clf.best_params_['kernel'])
        outer_clf = classifier.fit(VIS_train_outer, label_train_outer)
        
        # Validate the classifier
        score = outer_clf.score(VIS_test_outer, label_test_outer)
        clf_score.append(score)
    
    # find the best hyper-combo
    best_idx = clf_score.index(max(clf_score))
    
    C_best_VIS.append(C_best[best_idx])
    gamma_best_VIS.append(gamma_best[best_idx])
    kernel_best_VIS.append(kernel_best[best_idx])
    degree_best_VIS.append(degree_best[best_idx])
    print ('Best cost value:', C_best_VIS[c])
    print ('Best gamma:', gamma_best_VIS[c])
    print ('Best kernel:', kernel_best_VIS[c])
    print ('Best degree:', degree_best_VIS[c])
    
    classifier = SVC(class_weight='balanced',
                     random_state=c,
                     C=C_best[best_idx],
                     gamma=gamma_best[best_idx],
                     kernel=kernel_best[best_idx],
                     degree=degree_best[best_idx],
                    decision_function_shape='ovo')
    final_clf_VIS = classifier.fit(LakeVIS_train, label_train)
    
    # Test the classifier
    clf_pred_VIS[:,c] = final_clf_VIS.predict(LakeVIS_test)
    clf_score_VIS.append(f1_score(label_test, clf_pred_VIS[:,c], average='macro'))
    print(str(c)+' f-1 score for Lake VIS is: '+str(clf_score_VIS[c]))
    clf_recall_VIS[c,:] = recall_score(label_test, clf_pred_VIS[:,c], average=None)
    
    ### Ensembling VIS and DFC ###
    decision_function_both = np.stack((final_clf_DFC.decision_function(LakeDFC_test),final_clf_VIS.decision_function(LakeVIS_test)),axis=2)
    decision_function_both = np.mean(decision_function_both,axis=2)
    decisions = np.where(decision_function_both>0,0,1) #0:positive, 1:negative
    for s in range(test_parcel_num):
        predictions = []
        # loop through all the classifiers:
        for vc in range(len(votes)):
            index = decisions[s,vc]
            #index = decisions_DFC[s,vc]
            #predictions.append(votes[vc,index])
            #index = decisions_VIS[s,vc]
            predictions.append(votes[vc,index])
        
        # find the category with most votes
        clf_pred_Both[s,c] = int(st.mode(predictions))
        
    clf_recall_Both[c,:] = recall_score(label_test[:,0], clf_pred_Both[:,c], average=None)
    clf_score_Both.append(f1_score(label_test[:,0], clf_pred_Both[:,c], average='macro'))
    print(str(c)+' f-1 score for BOTH is: '+str(clf_score_Both[c]))

Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
0 f-1 score for Lake DFC is: 0.3771320054214791
Best cost value: 0.001
Best gamma: 1000.0
Best kernel: poly
Best degree: 2
0 f-1 score for Lake VIS is: 0.39303706217392226
0 f-1 score for BOTH is: 0.40861721983838917
Best cost value: 100000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
1 f-1 score for Lake DFC is: 0.3262236788552578
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
1 f-1 score for Lake VIS is: 0.38235158235158234
1 f-1 score for BOTH is: 0.4199988411331988
Best cost value: 100000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
2 f-1 score for Lake DFC is: 0.36667789426410113
Best cost value: 0.001
Best gamma: 1000.0
Best kernel: poly
Best degree: 2
2 f-1 score for Lake VIS is: 0.3921611721611722
2 f-1 score for BOTH is: 0.3789837420379292
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
3 f-1 score for Lake DFC is: 0.4080413394654881
Best cost 

Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
28 f-1 score for Lake VIS is: 0.43460904415692797
28 f-1 score for BOTH is: 0.43198644512110607
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
29 f-1 score for Lake DFC is: 0.38525732851633815
Best cost value: 0.1
Best gamma: 10
Best kernel: poly
Best degree: 4
29 f-1 score for Lake VIS is: 0.37637362637362637
29 f-1 score for BOTH is: 0.4222802057886091
Best cost value: 0.1
Best gamma: 10
Best kernel: poly
Best degree: 3
30 f-1 score for Lake DFC is: 0.3788873087665281
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
30 f-1 score for Lake VIS is: 0.47746327746327744
30 f-1 score for BOTH is: 0.4960677472256056
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
31 f-1 score for Lake DFC is: 0.36339220303506015
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
31 f-1 score for Lake VIS is: 0.4360154405461039
31 f-1

Best cost value: 0.1
Best gamma: 1000.0
Best kernel: poly
Best degree: 2
57 f-1 score for Lake DFC is: 0.4252454574903555
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
57 f-1 score for Lake VIS is: 0.4470539442490663
57 f-1 score for BOTH is: 0.4954692513585731
Best cost value: 0.001
Best gamma: 1000.0
Best kernel: poly
Best degree: 2
58 f-1 score for Lake DFC is: 0.4231471359130934
Best cost value: 1e-05
Best gamma: 1000.0
Best kernel: poly
Best degree: 2
58 f-1 score for Lake VIS is: 0.40088100149883366
58 f-1 score for BOTH is: 0.4782693077230892
Best cost value: 1e-05
Best gamma: 1000.0
Best kernel: poly
Best degree: 3
59 f-1 score for Lake DFC is: 0.38828864879285047
Best cost value: 0.1
Best gamma: 10
Best kernel: poly
Best degree: 3
59 f-1 score for Lake VIS is: 0.3931146849866062
59 f-1 score for BOTH is: 0.3802705684856714
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
60 f-1 score for Lake DFC is: 0.3693163978878264
Best cost

Best cost value: 1e-05
Best gamma: 1000.0
Best kernel: poly
Best degree: 2
85 f-1 score for Lake VIS is: 0.41862334525378003
85 f-1 score for BOTH is: 0.41986746389377
Best cost value: 0.1
Best gamma: 1000.0
Best kernel: poly
Best degree: 2
86 f-1 score for Lake DFC is: 0.4433214337562164
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
86 f-1 score for Lake VIS is: 0.41383314426792683
86 f-1 score for BOTH is: 0.4315970229013707
Best cost value: 0.1
Best gamma: 10
Best kernel: poly
Best degree: 4
87 f-1 score for Lake DFC is: 0.39043231863838374
Best cost value: 0.1
Best gamma: 10
Best kernel: poly
Best degree: 3
87 f-1 score for Lake VIS is: 0.417982816585539
87 f-1 score for BOTH is: 0.44825408424239255
Best cost value: 100000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
88 f-1 score for Lake DFC is: 0.3736280305809361
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
88 f-1 score for Lake VIS is: 0.41114005439446405
88 f-1 score

Best cost value: 10
Best gamma: 10
Best kernel: rbf
Best degree: 2
113 f-1 score for Lake VIS is: 0.40762339123683666
113 f-1 score for BOTH is: 0.47721395841696596
Best cost value: 1e-05
Best gamma: 1000.0
Best kernel: poly
Best degree: 3
114 f-1 score for Lake DFC is: 0.3812111544369609
Best cost value: 10
Best gamma: 10
Best kernel: rbf
Best degree: 2
114 f-1 score for Lake VIS is: 0.4014611578897293
114 f-1 score for BOTH is: 0.41127197116521746
Best cost value: 0.001
Best gamma: 1000.0
Best kernel: poly
Best degree: 2
115 f-1 score for Lake DFC is: 0.31338648745230596
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
115 f-1 score for Lake VIS is: 0.33486731133789954
115 f-1 score for BOTH is: 0.37026470937165357
Best cost value: 1e-05
Best gamma: 1000.0
Best kernel: poly
Best degree: 3
116 f-1 score for Lake DFC is: 0.3397421763881391
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
116 f-1 score for Lake VIS is: 0.468127636163

...

Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
870 f-1 score for Lake DFC is: 0.36749469797008005
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
870 f-1 score for Lake VIS is: 0.454106962377639
870 f-1 score for BOTH is: 0.44398854967873863
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
871 f-1 score for Lake DFC is: 0.34810396610677363
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
871 f-1 score for Lake VIS is: 0.5025241251047703
871 f-1 score for BOTH is: 0.4448454141985425
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
872 f-1 score for Lake DFC is: 0.4456709956709958
Best cost value: 1e-05
Best gamma: 1000.0
Best kernel: poly
Best degree: 2
872 f-1 score for Lake VIS is: 0.3751826386911646
872 f-1 score for BOTH is: 0.4018192163400061
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
873 f-1 score for Lake DFC is: 0.

Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
898 f-1 score for Lake DFC is: 0.3619948991377563
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
898 f-1 score for Lake VIS is: 0.3563557790265865
898 f-1 score for BOTH is: 0.33697233605155025
Best cost value: 0.001
Best gamma: 1000.0
Best kernel: poly
Best degree: 2
899 f-1 score for Lake DFC is: 0.4175329487815564
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
899 f-1 score for Lake VIS is: 0.4896025076697347
899 f-1 score for BOTH is: 0.45579256111292715
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
900 f-1 score for Lake DFC is: 0.3290654466028942
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
900 f-1 score for Lake VIS is: 0.36702154202154197
900 f-1 score for BOTH is: 0.4148096963606065
Best cost value: 0.001
Best gamma: 1000.0
Best kernel: poly
Best degree: 2
901 f-1 score for Lake DFC is: 0.37391061

Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
926 f-1 score for Lake DFC is: 0.4185734900020614
Best cost value: 1e-05
Best gamma: 1000.0
Best kernel: poly
Best degree: 2
926 f-1 score for Lake VIS is: 0.4224239056852455
926 f-1 score for BOTH is: 0.4955584424133846
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
927 f-1 score for Lake DFC is: 0.3186717730890663
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
927 f-1 score for Lake VIS is: 0.38417271309725404
927 f-1 score for BOTH is: 0.3570589639364518
Best cost value: 10
Best gamma: 10
Best kernel: rbf
Best degree: 2
928 f-1 score for Lake DFC is: 0.3512274624688812
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
928 f-1 score for Lake VIS is: 0.4841342393619118
928 f-1 score for BOTH is: 0.46418563842754973
Best cost value: 0.001
Best gamma: 1000.0
Best kernel: poly
Best degree: 2
929 f-1 score for Lake DFC is: 0.4047397606550026
B

Best cost value: 100000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
954 f-1 score for Lake DFC is: 0.45245596697209595
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
954 f-1 score for Lake VIS is: 0.45108549446117513
954 f-1 score for BOTH is: 0.4550816622545734
Best cost value: 1000.0
Best gamma: 10
Best kernel: rbf
Best degree: 2
955 f-1 score for Lake DFC is: 0.45442841100245474
Best cost value: 0.001
Best gamma: 1000.0
Best kernel: poly
Best degree: 2
955 f-1 score for Lake VIS is: 0.37034129891272755
955 f-1 score for BOTH is: 0.3997776289701756
Best cost value: 0.1
Best gamma: 10
Best kernel: poly
Best degree: 4
956 f-1 score for Lake DFC is: 0.41038901747002315
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
956 f-1 score for Lake VIS is: 0.4032841676246864
956 f-1 score for BOTH is: 0.493273873812848
Best cost value: 1000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
957 f-1 score for Lake DFC is: 0.3890863418753886

Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
982 f-1 score for Lake DFC is: 0.3642212687818916
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
982 f-1 score for Lake VIS is: 0.40234321515786053
982 f-1 score for BOTH is: 0.43128393699822276
Best cost value: 1e-05
Best gamma: 1000.0
Best kernel: poly
Best degree: 3
983 f-1 score for Lake DFC is: 0.35444996016424596
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
983 f-1 score for Lake VIS is: 0.39781418919654976
983 f-1 score for BOTH is: 0.44907815605265755
Best cost value: 100000.0
Best gamma: 0.1
Best kernel: rbf
Best degree: 2
984 f-1 score for Lake DFC is: 0.32210793940117244
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
984 f-1 score for Lake VIS is: 0.37845983045170534
984 f-1 score for BOTH is: 0.3471793550014312
Best cost value: 100000.0
Best gamma: 0.001
Best kernel: rbf
Best degree: 2
985 f-1 score for Lake DFC

In [49]:
# save the results
with open(net_enrich_dir+'/1000ClassifiersBalancedEnsembled_Full_2.pkl', 'wb') as f:
    pickle.dump([clf_test, sample_test,clf_pred_Both,clf_score_Both,clf_recall_Both,
                 clf_pred_DFC, clf_score_DFC, clf_recall_DFC, C_best_DFC, gamma_best_DFC, kernel_best_DFC,degree_best_DFC,
                 clf_pred_VIS, clf_score_VIS, clf_recall_VIS, C_best_VIS, gamma_best_VIS, kernel_best_VIS,degree_best_VIS], f)

In [50]:
# load the results
clf_test, sample_test,clf_pred_Both,clf_score_Both,clf_recall_Both, clf_pred_DFC, clf_score_DFC, clf_recall_DFC, C_best_DFC, gamma_best_DFC, kernel_best_DFC, degree_best_DFC, clf_pred_VIS, clf_score_VIS, clf_recall_VIS, C_best_VIS, gamma_best_VIS, kernel_best_VIS, degree_best_VIS = pickle.load(open(net_enrich_dir+'/1000ClassifiersBalancedEnsembled_Full_2.pkl',"rb"))


#### 1000 perm nulls

In [52]:
clf_num = 1000
parcel_num = 339
test_parcel_num = 102
net_num = 7

# Step0: Permutate the labels
np.random.seed(6666)
schaefer400_7Networks_perm = np.random.permutation(schaefer400_7Networks_mat)
#schaefer400_7Networks_perm = np.random.randint(low=1, high=8, size=parcel_num)

null_test = np.zeros((test_parcel_num,clf_num))
sample_test = np.zeros((test_parcel_num,clf_num))

null_pred_Both = np.zeros((test_parcel_num,clf_num))
null_score_Both = []
null_recall_Both = np.zeros((clf_num,net_num))

null_pred_DFC = np.zeros((test_parcel_num,clf_num))
null_score_DFC = []
null_recall_DFC = np.zeros((clf_num,net_num))

null_pred_VIS = np.zeros((test_parcel_num,clf_num))
null_score_VIS = []
null_recall_VIS = np.zeros((clf_num,net_num))
for c in range(clf_num):

    # Step1: Split 70% for train and 30% for test
    train, test = train_test_split(list(range(parcel_num)), test_size=0.3, stratify=schaefer400_7Networks_perm,random_state=c+1000)
    label_train = schaefer400_7Networks_perm[train]
    label_test = schaefer400_7Networks_perm[test]
    # save the permutated lable
    null_test[:,c] = label_test[:,0]
    #null_test[:,c] = label_test
    # save the ID of the tested parcels
    sample_test[:,c] = test

    LakeDFC_train = np.transpose(LakeDFC_schaefer400_mat[:,train])
    LakeDFC_test = np.transpose(LakeDFC_schaefer400_mat[:,test])

    LakeVIS_train = np.transpose(LakeVIS_schaefer400_mat[:,train])
    LakeVIS_test = np.transpose(LakeVIS_schaefer400_mat[:,test])


    ### Lake DFC ###
    # Step2: Random draw a hyper-combo then train the classifer
    # find the best hyper-combo
    np.random.seed(c+1000)
    best_idx = np.random.randint(1000, size=1)[0]

    # Step3: Train the classifier
    classifier = SVC(class_weight='balanced',
                     random_state=c+1000,
                     C=C_best_DFC[best_idx],
                     gamma=gamma_best_DFC[best_idx],
                     degree=degree_best_DFC[best_idx],
                     kernel=kernel_best_DFC[best_idx],
                     decision_function_shape='ovo')
    final_clf_DFC = classifier.fit(LakeDFC_train, label_train)

    # Test the classifier
    null_pred_DFC[:,c] = final_clf_DFC.predict(LakeDFC_test)
    null_score_DFC.append(f1_score(label_test, null_pred_DFC[:,c], average='macro'))
    print(str(c)+' f-1 score for Lake DFC is: '+str(null_score_DFC[c]))
    null_recall_DFC[c,:] = recall_score(label_test, null_pred_DFC[:,c], average=None)


    ### Lake VIS ###
    # Step2: Random draw a hyper-combo then train the classifer
    # find the best hyper-combo
    np.random.seed(c+1000)
    best_idx = np.random.randint(1000, size=1)[0]


    # Step3: Train the classifier
    classifier = SVC(class_weight='balanced',
                     random_state=c+1000,
                     C=C_best_VIS[best_idx],
                     gamma=gamma_best_VIS[best_idx],
                     degree=degree_best_VIS[best_idx],
                     kernel=kernel_best_VIS[best_idx],
                     decision_function_shape='ovo')
    final_clf_VIS = classifier.fit(LakeVIS_train, label_train)

    # Test the classifier
    null_pred_VIS[:,c] = final_clf_VIS.predict(LakeVIS_test)
    null_score_VIS.append(f1_score(label_test, null_pred_VIS[:,c], average='macro'))
    print(str(c)+' f-1 score for Lake VIS is: '+str(null_score_VIS[c]))
    null_recall_VIS[c,:] = recall_score(label_test, null_pred_VIS[:,c], average=None)
    
    ### Ensembling VIS and DFC ###
    decision_function_both = np.stack((final_clf_DFC.decision_function(LakeDFC_test),final_clf_VIS.decision_function(LakeVIS_test)),axis=2)
    decision_function_both = np.mean(decision_function_both,axis=2)
    decisions = np.where(decision_function_both>0,0,1) #0:positive, 1:negative
    for s in range(test_parcel_num):
        predictions = []
        # loop through all the classifiers:
        for vc in range(len(votes)):
            index = decisions[s,vc]
            #index = decisions_DFC[s,vc]
            #predictions.append(votes[vc,index])
            #index = decisions_VIS[s,vc]
            predictions.append(votes[vc,index])
        
        # find the category with most votes
        null_pred_Both[s,c] = int(st.mode(predictions))
        
    null_recall_Both[c,:] = recall_score(label_test, null_pred_Both[:,c], average=None)
    null_score_Both.append(f1_score(label_test, null_pred_Both[:,c], average='macro'))
    print(str(c)+' f-1 score for BOTH is: '+str(null_score_Both[c]))

0 f-1 score for Lake DFC is: 0.148848549014662
0 f-1 score for Lake VIS is: 0.15255256910604958
0 f-1 score for BOTH is: 0.13877691772428616
1 f-1 score for Lake DFC is: 0.09916792202506487
1 f-1 score for Lake VIS is: 0.09145072170282255
1 f-1 score for BOTH is: 0.08007421150278295
2 f-1 score for Lake DFC is: 0.07674954457563153
2 f-1 score for Lake VIS is: 0.07736044657097288
2 f-1 score for BOTH is: 0.05161637304494448
3 f-1 score for Lake DFC is: 0.17319681548160212
3 f-1 score for Lake VIS is: 0.13467167849395484
3 f-1 score for BOTH is: 0.1844109851006403
4 f-1 score for Lake DFC is: 0.1474625599625599
4 f-1 score for Lake VIS is: 0.11819396390824963
4 f-1 score for BOTH is: 0.09905919814734403
5 f-1 score for Lake DFC is: 0.18563335432349656
5 f-1 score for Lake VIS is: 0.12356657711348787
5 f-1 score for BOTH is: 0.14693292188754983
6 f-1 score for Lake DFC is: 0.09739402373635013
6 f-1 score for Lake VIS is: 0.03871467286101433
6 f-1 score for BOTH is: 0.07740023134759977
7 f

58 f-1 score for Lake DFC is: 0.1330802530802531
58 f-1 score for Lake VIS is: 0.10121801870359312
58 f-1 score for BOTH is: 0.10993285993285994
59 f-1 score for Lake DFC is: 0.09007227228642838
59 f-1 score for Lake VIS is: 0.12102361983161544
59 f-1 score for BOTH is: 0.10502933360076216
60 f-1 score for Lake DFC is: 0.17703689167974884
60 f-1 score for Lake VIS is: 0.08307127882599581
60 f-1 score for BOTH is: 0.16638572312041702
61 f-1 score for Lake DFC is: 0.15419722326343718
61 f-1 score for Lake VIS is: 0.1532258064516129
61 f-1 score for BOTH is: 0.13487464916036343
62 f-1 score for Lake DFC is: 0.16044738330452618
62 f-1 score for Lake VIS is: 0.13342596101216792
62 f-1 score for BOTH is: 0.14466565977066367
63 f-1 score for Lake DFC is: 0.15206590913112653
63 f-1 score for Lake VIS is: 0.09657820440429135
63 f-1 score for BOTH is: 0.16802068187741206
64 f-1 score for Lake DFC is: 0.13570792587126526
64 f-1 score for Lake VIS is: 0.14049219780154804
64 f-1 score for BOTH is: 

119 f-1 score for Lake DFC is: 0.1268552674706957
119 f-1 score for Lake VIS is: 0.10888888888888888
119 f-1 score for BOTH is: 0.13454824983064187
120 f-1 score for Lake DFC is: 0.12553460514640638
120 f-1 score for Lake VIS is: 0.11857409283879872
120 f-1 score for BOTH is: 0.12217930536896056
121 f-1 score for Lake DFC is: 0.1715046792041143
121 f-1 score for Lake VIS is: 0.1781390818068998
121 f-1 score for BOTH is: 0.12011787473610133
122 f-1 score for Lake DFC is: 0.1526624832777677
122 f-1 score for Lake VIS is: 0.07262500109624935
122 f-1 score for BOTH is: 0.13830175832368014
123 f-1 score for Lake DFC is: 0.09949921187612445
123 f-1 score for Lake VIS is: 0.11862708719851578
123 f-1 score for BOTH is: 0.10249841032449727
124 f-1 score for Lake DFC is: 0.08683677433677432
124 f-1 score for Lake VIS is: 0.11064133941892854
124 f-1 score for BOTH is: 0.08128624492619155
125 f-1 score for Lake DFC is: 0.09936151065183323
125 f-1 score for Lake VIS is: 0.07358574452107518
125 f-1 

175 f-1 score for Lake DFC is: 0.1735014806035638
175 f-1 score for Lake VIS is: 0.1263548651783946
175 f-1 score for BOTH is: 0.13603521498258342
176 f-1 score for Lake DFC is: 0.11854464862868223
176 f-1 score for Lake VIS is: 0.08864999453234748
176 f-1 score for BOTH is: 0.10386249905700706
177 f-1 score for Lake DFC is: 0.0738590807556325
177 f-1 score for Lake VIS is: 0.1164076492095997
177 f-1 score for BOTH is: 0.051905782555937356
178 f-1 score for Lake DFC is: 0.09972227329852015
178 f-1 score for Lake VIS is: 0.09459283236260295
178 f-1 score for BOTH is: 0.1177933177933178
179 f-1 score for Lake DFC is: 0.143534833288154
179 f-1 score for Lake VIS is: 0.14731969342761028
179 f-1 score for BOTH is: 0.14367398474541332
180 f-1 score for Lake DFC is: 0.11448245965052688
180 f-1 score for Lake VIS is: 0.10619803476946334
180 f-1 score for BOTH is: 0.09551612180922527
181 f-1 score for Lake DFC is: 0.11258493140846083
181 f-1 score for Lake VIS is: 0.10516304505318448
181 f-1 sc

231 f-1 score for BOTH is: 0.06674806674806674
232 f-1 score for Lake DFC is: 0.14593998551371823
232 f-1 score for Lake VIS is: 0.12576689047277284
232 f-1 score for BOTH is: 0.12585721157149726
233 f-1 score for Lake DFC is: 0.142961711318021
233 f-1 score for Lake VIS is: 0.09161076416397543
233 f-1 score for BOTH is: 0.12476224141123397
234 f-1 score for Lake DFC is: 0.14161128066787357
234 f-1 score for Lake VIS is: 0.04467678544285931
234 f-1 score for BOTH is: 0.13497273650075006
235 f-1 score for Lake DFC is: 0.12128331247932578
235 f-1 score for Lake VIS is: 0.10572228891556623
235 f-1 score for BOTH is: 0.14699099841956986
236 f-1 score for Lake DFC is: 0.116656641945859
236 f-1 score for Lake VIS is: 0.11448691604788368
236 f-1 score for BOTH is: 0.14214582802818096
237 f-1 score for Lake DFC is: 0.07591070616280701
237 f-1 score for Lake VIS is: 0.08543708543708543
237 f-1 score for BOTH is: 0.10296195033037138
238 f-1 score for Lake DFC is: 0.09966170003763988
238 f-1 scor

...

745 f-1 score for Lake VIS is: 0.14549801475267315
745 f-1 score for BOTH is: 0.11173019078432407
746 f-1 score for Lake DFC is: 0.17106764535335964
746 f-1 score for Lake VIS is: 0.1596004409669006
746 f-1 score for BOTH is: 0.10436530887658708
747 f-1 score for Lake DFC is: 0.08623449379751902
747 f-1 score for Lake VIS is: 0.09322141956835625
747 f-1 score for BOTH is: 0.07070364082248633
748 f-1 score for Lake DFC is: 0.11123930217248189
748 f-1 score for Lake VIS is: 0.13062983011969825
748 f-1 score for BOTH is: 0.11243306683943158
749 f-1 score for Lake DFC is: 0.16258698694788923
749 f-1 score for Lake VIS is: 0.10802640241155885
749 f-1 score for BOTH is: 0.12290568932168346
750 f-1 score for Lake DFC is: 0.14999282522287344
750 f-1 score for Lake VIS is: 0.16703772418058133
750 f-1 score for BOTH is: 0.16927775158646455
751 f-1 score for Lake DFC is: 0.18136335209505944
751 f-1 score for Lake VIS is: 0.12671869692546384
751 f-1 score for BOTH is: 0.11194075132114822
752 f-1 s

803 f-1 score for Lake DFC is: 0.14869703288698793
803 f-1 score for Lake VIS is: 0.08736935298478123
803 f-1 score for BOTH is: 0.14936666274901572
804 f-1 score for Lake DFC is: 0.12913626056483202
804 f-1 score for Lake VIS is: 0.10105409685241618
804 f-1 score for BOTH is: 0.10493802263988598
805 f-1 score for Lake DFC is: 0.14229249011857706
805 f-1 score for Lake VIS is: 0.06344939409455538
805 f-1 score for BOTH is: 0.13662337662337662
806 f-1 score for Lake DFC is: 0.08756017026693717
806 f-1 score for Lake VIS is: 0.11624649859943978
806 f-1 score for BOTH is: 0.08668610301263362
807 f-1 score for Lake DFC is: 0.12053686193221076
807 f-1 score for Lake VIS is: 0.08660504975021104
807 f-1 score for BOTH is: 0.1077152600170503
808 f-1 score for Lake DFC is: 0.16190019383433957
808 f-1 score for Lake VIS is: 0.10180212435851534
808 f-1 score for BOTH is: 0.07372502937720328
809 f-1 score for Lake DFC is: 0.11655638054157418
809 f-1 score for Lake VIS is: 0.10232323232323234
809 f

859 f-1 score for Lake DFC is: 0.16190667010142065
859 f-1 score for Lake VIS is: 0.10041900209967436
859 f-1 score for BOTH is: 0.11578629361880595
860 f-1 score for Lake DFC is: 0.1848507180650038
860 f-1 score for Lake VIS is: 0.12469966619126284
860 f-1 score for BOTH is: 0.14021696557410843
861 f-1 score for Lake DFC is: 0.15290306764227032
861 f-1 score for Lake VIS is: 0.1663264880290821
861 f-1 score for BOTH is: 0.14706801135372566
862 f-1 score for Lake DFC is: 0.13961190789432845
862 f-1 score for Lake VIS is: 0.14905934004807597
862 f-1 score for BOTH is: 0.11012400097954093
863 f-1 score for Lake DFC is: 0.11102911064091188
863 f-1 score for Lake VIS is: 0.06104494583757256
863 f-1 score for BOTH is: 0.14344038149203236
864 f-1 score for Lake DFC is: 0.11736464019072715
864 f-1 score for Lake VIS is: 0.10935374149659867
864 f-1 score for BOTH is: 0.09508174752077192
865 f-1 score for Lake DFC is: 0.09482240861551205
865 f-1 score for Lake VIS is: 0.1251863847771776
865 f-1

919 f-1 score for Lake VIS is: 0.12737615166186594
919 f-1 score for BOTH is: 0.11489777891966675
920 f-1 score for Lake DFC is: 0.09327863771878493
920 f-1 score for Lake VIS is: 0.13219472109223665
920 f-1 score for BOTH is: 0.15402752009894868
921 f-1 score for Lake DFC is: 0.13283052351375335
921 f-1 score for Lake VIS is: 0.09132001135177178
921 f-1 score for BOTH is: 0.09925610875723156
922 f-1 score for Lake DFC is: 0.12238767159387529
922 f-1 score for Lake VIS is: 0.11064051505334185
922 f-1 score for BOTH is: 0.13481458388290687
923 f-1 score for Lake DFC is: 0.14775717540694502
923 f-1 score for Lake VIS is: 0.08340871993623418
923 f-1 score for BOTH is: 0.11377143750242914
924 f-1 score for Lake DFC is: 0.13626373626373628
924 f-1 score for Lake VIS is: 0.08449772388391315
924 f-1 score for BOTH is: 0.14167499167499167
925 f-1 score for Lake DFC is: 0.09226952825783662
925 f-1 score for Lake VIS is: 0.1705450665114531
925 f-1 score for BOTH is: 0.11244921720871455
926 f-1 s

975 f-1 score for Lake DFC is: 0.13877321990633756
975 f-1 score for Lake VIS is: 0.12480902580281462
975 f-1 score for BOTH is: 0.12040628217098806
976 f-1 score for Lake DFC is: 0.09612164050679695
976 f-1 score for Lake VIS is: 0.09694971694971695
976 f-1 score for BOTH is: 0.1021572611682563
977 f-1 score for Lake DFC is: 0.14176458462172747
977 f-1 score for Lake VIS is: 0.10052499260573795
977 f-1 score for BOTH is: 0.14213353222336128
978 f-1 score for Lake DFC is: 0.09736819283617314
978 f-1 score for Lake VIS is: 0.06242107486009924
978 f-1 score for BOTH is: 0.1024740228404385
979 f-1 score for Lake DFC is: 0.07565091912918
979 f-1 score for Lake VIS is: 0.07388503289431639
979 f-1 score for BOTH is: 0.0740618399505809
980 f-1 score for Lake DFC is: 0.12611247537355913
980 f-1 score for Lake VIS is: 0.06090808416389811
980 f-1 score for BOTH is: 0.11273245027397853
981 f-1 score for Lake DFC is: 0.1783932940765239
981 f-1 score for Lake VIS is: 0.11196581196581197
981 f-1 sco

In [53]:
# save the results
with open(net_enrich_dir+'/1000NullsBalancedPermEnsembeled_Full.pkl', 'wb') as f:
    pickle.dump([null_test, sample_test, null_pred_Both, null_score_Both, null_recall_Both,
                 null_pred_DFC, null_score_DFC, null_recall_DFC,
                 null_pred_VIS, null_score_VIS, null_recall_VIS], f)

In [54]:
# load the results
null_test, sample_test, null_pred_Both, null_score_Both, null_recall_Both, null_pred_DFC, null_score_DFC, null_recall_DFC, null_pred_VIS, null_score_VIS, null_recall_VIS = pickle.load(open(net_enrich_dir+'/1000NullsBalancedPermEnsembeled_Full.pkl',"rb"))

#### 1000 spin nulls

In [56]:
clf_num = 1000
parcel_num = 339
test_parcel_num = 102
net_num = 7

null_test = np.zeros((test_parcel_num,clf_num))
sample_test = np.zeros((test_parcel_num,clf_num))

null_pred_Both = np.zeros((test_parcel_num,clf_num))
null_score_Both = []
null_recall_Both = np.zeros((clf_num,net_num))

null_pred_DFC = np.zeros((test_parcel_num,clf_num))
null_score_DFC = []
null_recall_DFC = np.zeros((clf_num,net_num))

null_pred_VIS = np.zeros((test_parcel_num,clf_num))
null_score_VIS = []
null_recall_VIS = np.zeros((clf_num,net_num))
for c in range(clf_num):
    
    # Step0: Get the spin-tested label
    schaefer400_7Networks_perm = Null_7NetLabel_mat[:,c]

    # Step1: Split 70% for train and 30% for test
    train, test = train_test_split(list(range(parcel_num)), test_size=0.3, stratify=schaefer400_7Networks_perm,random_state=c+1000)
    label_train = schaefer400_7Networks_perm[train]
    label_test = schaefer400_7Networks_perm[test]
    # save the permutated lable
    #null_test[:,c] = label_test[:,0]
    null_test[:,c] = label_test
    # save the ID of the tested parcels
    sample_test[:,c] = test

    LakeDFC_train = np.transpose(LakeDFC_schaefer400_mat[:,train])
    LakeDFC_test = np.transpose(LakeDFC_schaefer400_mat[:,test])

    LakeVIS_train = np.transpose(LakeVIS_schaefer400_mat[:,train])
    LakeVIS_test = np.transpose(LakeVIS_schaefer400_mat[:,test])


    ### Lake DFC ###
    # Step2: Random draw a hyper-combo then train the classifer
    # find the best hyper-combo
    np.random.seed(c+1000)
    best_idx = np.random.randint(1000, size=1)[0]

    # Step3: Train the classifier
    classifier = SVC(class_weight='balanced',
                     random_state=c+1000,
                     C=C_best_DFC[best_idx],
                     gamma=gamma_best_DFC[best_idx],
                     degree=degree_best_DFC[best_idx],
                     kernel=kernel_best_DFC[best_idx],
                     decision_function_shape='ovo')
    final_clf_DFC = classifier.fit(LakeDFC_train, label_train)

    # Test the classifier
    null_pred_DFC[:,c] = final_clf_DFC.predict(LakeDFC_test)
    null_score_DFC.append(f1_score(label_test, null_pred_DFC[:,c], average='macro'))
    print(str(c)+' f-1 score for Lake DFC is: '+str(null_score_DFC[c]))
    null_recall_DFC[c,:] = recall_score(label_test, null_pred_DFC[:,c], average=None)


    ### Lake VIS ###
    # Step2: Random draw a hyper-combo then train the classifer
    # find the best hyper-combo
    np.random.seed(c+1000)
    best_idx = np.random.randint(1000, size=1)[0]


    # Step3: Train the classifier
    classifier = SVC(class_weight='balanced',
                     random_state=c+1000,
                     C=C_best_VIS[best_idx],
                     gamma=gamma_best_VIS[best_idx],
                     degree=degree_best_VIS[best_idx],
                     kernel=kernel_best_VIS[best_idx],
                     decision_function_shape='ovo')
    final_clf_VIS = classifier.fit(LakeVIS_train, label_train)

    # Test the classifier
    null_pred_VIS[:,c] = final_clf_VIS.predict(LakeVIS_test)
    null_score_VIS.append(f1_score(label_test, null_pred_VIS[:,c], average='macro'))
    print(str(c)+' f-1 score for Lake VIS is: '+str(null_score_VIS[c]))
    null_recall_VIS[c,:] = recall_score(label_test, null_pred_VIS[:,c], average=None)
    
    ### Ensembling VIS and DFC ###
    decision_function_both = np.stack((final_clf_DFC.decision_function(LakeDFC_test),final_clf_VIS.decision_function(LakeVIS_test)),axis=2)
    decision_function_both = np.mean(decision_function_both,axis=2)
    decisions = np.where(decision_function_both>0,0,1) #0:positive, 1:negative
    for s in range(test_parcel_num):
        predictions = []
        # loop through all the classifiers:
        for vc in range(len(votes)):
            index = decisions[s,vc]
            #index = decisions_DFC[s,vc]
            #predictions.append(votes[vc,index])
            #index = decisions_VIS[s,vc]
            predictions.append(votes[vc,index])
        
        # find the category with most votes
        null_pred_Both[s,c] = int(st.mode(predictions))
        
    null_recall_Both[c,:] = recall_score(label_test, null_pred_Both[:,c], average=None)
    null_score_Both.append(f1_score(label_test, null_pred_Both[:,c], average='macro'))
    print(str(c)+' f-1 score for BOTH is: '+str(null_score_Both[c]))

0 f-1 score for Lake DFC is: 0.23468082617018787
0 f-1 score for Lake VIS is: 0.2500369713102632
0 f-1 score for BOTH is: 0.2406203060934519
1 f-1 score for Lake DFC is: 0.3449388949388949
1 f-1 score for Lake VIS is: 0.32892112447516003
1 f-1 score for BOTH is: 0.34262454196009007
2 f-1 score for Lake DFC is: 0.265763710870724
2 f-1 score for Lake VIS is: 0.19459278327659263
2 f-1 score for BOTH is: 0.24879573870257102
3 f-1 score for Lake DFC is: 0.25401465453137184
3 f-1 score for Lake VIS is: 0.3157098939707636
3 f-1 score for BOTH is: 0.2565329002807029
4 f-1 score for Lake DFC is: 0.2814030740501329
4 f-1 score for Lake VIS is: 0.3251580104559037
4 f-1 score for BOTH is: 0.3197197109202186
5 f-1 score for Lake DFC is: 0.23514516500784177
5 f-1 score for Lake VIS is: 0.13952994642649813
5 f-1 score for BOTH is: 0.18284086284086284
6 f-1 score for Lake DFC is: 0.24550044984255512
6 f-1 score for Lake VIS is: 0.19793347092081734
6 f-1 score for BOTH is: 0.2101571164864413
7 f-1 scor

59 f-1 score for Lake VIS is: 0.37249462796575256
59 f-1 score for BOTH is: 0.3714446841334199
60 f-1 score for Lake DFC is: 0.20860730411421655
60 f-1 score for Lake VIS is: 0.23081278337849048
60 f-1 score for BOTH is: 0.24469087143859178
61 f-1 score for Lake DFC is: 0.20357870308834675
61 f-1 score for Lake VIS is: 0.24565149136577707
61 f-1 score for BOTH is: 0.19880315762668702
62 f-1 score for Lake DFC is: 0.26549916532938606
62 f-1 score for Lake VIS is: 0.2477919040131022
62 f-1 score for BOTH is: 0.22476239671190582
63 f-1 score for Lake DFC is: 0.17007658807402432
63 f-1 score for Lake VIS is: 0.21074965648883343
63 f-1 score for BOTH is: 0.2281914056914057
64 f-1 score for Lake DFC is: 0.23907632799822956
64 f-1 score for Lake VIS is: 0.1469138951536064
64 f-1 score for BOTH is: 0.24470072442422672
65 f-1 score for Lake DFC is: 0.23090247716174384
65 f-1 score for Lake VIS is: 0.2142920288281319
65 f-1 score for BOTH is: 0.2417054939867489
66 f-1 score for Lake DFC is: 0.19

119 f-1 score for Lake DFC is: 0.2781519435654774
119 f-1 score for Lake VIS is: 0.23033823319537605
119 f-1 score for BOTH is: 0.32768765400897204
120 f-1 score for Lake DFC is: 0.3155862775217614
120 f-1 score for Lake VIS is: 0.34903951251777343
120 f-1 score for BOTH is: 0.31864036244172633
121 f-1 score for Lake DFC is: 0.19520030234315947
121 f-1 score for Lake VIS is: 0.2346198534830756
121 f-1 score for BOTH is: 0.23455972132442723
122 f-1 score for Lake DFC is: 0.23050532458366563
122 f-1 score for Lake VIS is: 0.22847600578693014
122 f-1 score for BOTH is: 0.24657167076414083
123 f-1 score for Lake DFC is: 0.243516182482785
123 f-1 score for Lake VIS is: 0.3342497273508946
123 f-1 score for BOTH is: 0.2997184414606017
124 f-1 score for Lake DFC is: 0.26825604464103764
124 f-1 score for Lake VIS is: 0.2538830696725434
124 f-1 score for BOTH is: 0.25402436654808574
125 f-1 score for Lake DFC is: 0.19025528285110166
125 f-1 score for Lake VIS is: 0.23868402533427163
125 f-1 scor

175 f-1 score for Lake DFC is: 0.2403327234982568
175 f-1 score for Lake VIS is: 0.26723721067374323
175 f-1 score for BOTH is: 0.23667786245659503
176 f-1 score for Lake DFC is: 0.2386145169530263
176 f-1 score for Lake VIS is: 0.16582546997293543
176 f-1 score for BOTH is: 0.22756670520645678
177 f-1 score for Lake DFC is: 0.21249813563478162
177 f-1 score for Lake VIS is: 0.2755778334725703
177 f-1 score for BOTH is: 0.2416803717824126
178 f-1 score for Lake DFC is: 0.23781045599017642
178 f-1 score for Lake VIS is: 0.20934068046871093
178 f-1 score for BOTH is: 0.20265924551638834
179 f-1 score for Lake DFC is: 0.3145494822936509
179 f-1 score for Lake VIS is: 0.21093738950881807
179 f-1 score for BOTH is: 0.3414419888104098
180 f-1 score for Lake DFC is: 0.2075741407674181
180 f-1 score for Lake VIS is: 0.2537787558326711
180 f-1 score for BOTH is: 0.24802440012524046
181 f-1 score for Lake DFC is: 0.2827481869940959
181 f-1 score for Lake VIS is: 0.2943670531640456
181 f-1 score 

234 f-1 score for Lake DFC is: 0.23543089577361304
234 f-1 score for Lake VIS is: 0.30410800000109406
234 f-1 score for BOTH is: 0.2671747688426849
235 f-1 score for Lake DFC is: 0.1503379661274398
235 f-1 score for Lake VIS is: 0.1753036480585093
235 f-1 score for BOTH is: 0.16528682933088915
236 f-1 score for Lake DFC is: 0.1855310568493636
236 f-1 score for Lake VIS is: 0.2382615245259662
236 f-1 score for BOTH is: 0.2078510651909667
237 f-1 score for Lake DFC is: 0.27265834563490465
237 f-1 score for Lake VIS is: 0.2158631263336761
237 f-1 score for BOTH is: 0.2490890062318634
238 f-1 score for Lake DFC is: 0.20281357822587207
238 f-1 score for Lake VIS is: 0.2517906745818785
238 f-1 score for BOTH is: 0.23809754096932487
239 f-1 score for Lake DFC is: 0.2097007325278484
239 f-1 score for Lake VIS is: 0.30940683619255044
239 f-1 score for BOTH is: 0.3401293586655144
240 f-1 score for Lake DFC is: 0.19944114363310295
240 f-1 score for Lake VIS is: 0.26605362424690154
240 f-1 score f

...

759 f-1 score for Lake VIS is: 0.24378778777275018
759 f-1 score for BOTH is: 0.2002463054187192
760 f-1 score for Lake DFC is: 0.22260217260217255
760 f-1 score for Lake VIS is: 0.22160919540229887
760 f-1 score for BOTH is: 0.24721484283048037
761 f-1 score for Lake DFC is: 0.30162195031927347
761 f-1 score for Lake VIS is: 0.20792958857474983
761 f-1 score for BOTH is: 0.25404928053092585
762 f-1 score for Lake DFC is: 0.224898251683966
762 f-1 score for Lake VIS is: 0.17860867423892637
762 f-1 score for BOTH is: 0.17789582599656553
763 f-1 score for Lake DFC is: 0.24709523423809135
763 f-1 score for Lake VIS is: 0.21770717594700764
763 f-1 score for BOTH is: 0.26262487164742804
764 f-1 score for Lake DFC is: 0.18958810353409655
764 f-1 score for Lake VIS is: 0.24227485044360994
764 f-1 score for BOTH is: 0.2365475886325926
765 f-1 score for Lake DFC is: 0.1854141229141229
765 f-1 score for Lake VIS is: 0.23870688626327727
765 f-1 score for BOTH is: 0.24255002615646096
766 f-1 score

818 f-1 score for Lake DFC is: 0.22383783922925898
818 f-1 score for Lake VIS is: 0.2036059436059436
818 f-1 score for BOTH is: 0.21164177866268458
819 f-1 score for Lake DFC is: 0.22072482839289562
819 f-1 score for Lake VIS is: 0.22872927544704438
819 f-1 score for BOTH is: 0.17717532467532468
820 f-1 score for Lake DFC is: 0.2814990577921613
820 f-1 score for Lake VIS is: 0.2962061960996351
820 f-1 score for BOTH is: 0.27127407578535395
821 f-1 score for Lake DFC is: 0.2847602010392708
821 f-1 score for Lake VIS is: 0.2811460235132318
821 f-1 score for BOTH is: 0.3440565699964196
822 f-1 score for Lake DFC is: 0.24783263747585577
822 f-1 score for Lake VIS is: 0.18227018875488757
822 f-1 score for BOTH is: 0.26889623718460515
823 f-1 score for Lake DFC is: 0.2831646979720001
823 f-1 score for Lake VIS is: 0.2649855902296009
823 f-1 score for BOTH is: 0.29897186147186144
824 f-1 score for Lake DFC is: 0.25069160997732426
824 f-1 score for Lake VIS is: 0.2515010739565554
824 f-1 score

876 f-1 score for Lake VIS is: 0.22794293350436218
876 f-1 score for BOTH is: 0.2647481201646346
877 f-1 score for Lake DFC is: 0.1960954154401718
877 f-1 score for Lake VIS is: 0.20325700615174297
877 f-1 score for BOTH is: 0.239850142955733
878 f-1 score for Lake DFC is: 0.18028140727255137
878 f-1 score for Lake VIS is: 0.24092044627698275
878 f-1 score for BOTH is: 0.19528809444775833
879 f-1 score for Lake DFC is: 0.20093381817752295
879 f-1 score for Lake VIS is: 0.1924775405393265
879 f-1 score for BOTH is: 0.2270863901116002
880 f-1 score for Lake DFC is: 0.21911306475297979
880 f-1 score for Lake VIS is: 0.208411737900902
880 f-1 score for BOTH is: 0.2644004461688057
881 f-1 score for Lake DFC is: 0.20491702200847625
881 f-1 score for Lake VIS is: 0.2103452625069166
881 f-1 score for BOTH is: 0.20953456651477795
882 f-1 score for Lake DFC is: 0.17166249899169153
882 f-1 score for Lake VIS is: 0.17887050338687138
882 f-1 score for BOTH is: 0.2247903490400489
883 f-1 score for L

933 f-1 score for Lake VIS is: 0.21492391980196857
933 f-1 score for BOTH is: 0.21514983915570857
934 f-1 score for Lake DFC is: 0.21959738660505926
934 f-1 score for Lake VIS is: 0.20161614781413925
934 f-1 score for BOTH is: 0.1980636481761417
935 f-1 score for Lake DFC is: 0.2814312453502997
935 f-1 score for Lake VIS is: 0.25314319667260843
935 f-1 score for BOTH is: 0.28408473386047656
936 f-1 score for Lake DFC is: 0.18628539516493098
936 f-1 score for Lake VIS is: 0.22878625134264233
936 f-1 score for BOTH is: 0.23842946269416856
937 f-1 score for Lake DFC is: 0.23400412951104196
937 f-1 score for Lake VIS is: 0.24470786185071902
937 f-1 score for BOTH is: 0.25209648941900464
938 f-1 score for Lake DFC is: 0.13590327561732102
938 f-1 score for Lake VIS is: 0.2184166981159462
938 f-1 score for BOTH is: 0.19408312498878386
939 f-1 score for Lake DFC is: 0.19635470349756062
939 f-1 score for Lake VIS is: 0.2511100803957947
939 f-1 score for BOTH is: 0.24511169149917422
940 f-1 scor

990 f-1 score for Lake VIS is: 0.2409836865096247
990 f-1 score for BOTH is: 0.2926603895795167
991 f-1 score for Lake DFC is: 0.1562603355897074
991 f-1 score for Lake VIS is: 0.22820344745810583
991 f-1 score for BOTH is: 0.19321635536805812
992 f-1 score for Lake DFC is: 0.21039445374216398
992 f-1 score for Lake VIS is: 0.21555204597032826
992 f-1 score for BOTH is: 0.23979187420332698
993 f-1 score for Lake DFC is: 0.22717684228096677
993 f-1 score for Lake VIS is: 0.30054872028556245
993 f-1 score for BOTH is: 0.27567619884693056
994 f-1 score for Lake DFC is: 0.19880722426778472
994 f-1 score for Lake VIS is: 0.23419466764414504
994 f-1 score for BOTH is: 0.2425960668331301
995 f-1 score for Lake DFC is: 0.27957536099950964
995 f-1 score for Lake VIS is: 0.1916042858344264
995 f-1 score for BOTH is: 0.2376752414874981
996 f-1 score for Lake DFC is: 0.28434485455122605
996 f-1 score for Lake VIS is: 0.23935323935323938
996 f-1 score for BOTH is: 0.2812566550613494
997 f-1 score f

In [57]:
# save the results
with open(net_enrich_dir+'/1000NullsBalancedSpinEnsembeled_Full_2.pkl', 'wb') as f:
    pickle.dump([null_test, sample_test, null_pred_Both, null_score_Both, null_recall_Both,
                 null_pred_DFC, null_score_DFC, null_recall_DFC,
                 null_pred_VIS, null_score_VIS, null_recall_VIS], f)

In [58]:
# load the results
null_test, sample_test, null_pred_Both, null_score_Both, null_recall_Both, null_pred_DFC, null_score_DFC, null_recall_DFC, null_pred_VIS, null_score_VIS, null_recall_VIS = pickle.load(open(net_enrich_dir+'/1000NullsBalancedSpinEnsembeled_Full_2.pkl',"rb"))